In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mplsoccer import Pitch
import os
import ast
from unidecode import unidecode
from scipy.spatial import ConvexHull


In [149]:
df=pd.read_csv(r"E:\Ai_com\match_enriched\match_3925603.csv")

In [150]:
df['type_name'].unique()

array(['Starting XI', 'Half Start', 'Pass', 'Ball Receipt*', 'Carry',
       'Dribble', 'Duel', 'Clearance', 'Ball Recovery', 'Block',
       'Pressure', 'Dispossessed', 'Miscontrol', 'Goal Keeper',
       'Foul Committed', 'Foul Won', 'Injury Stoppage', 'Dribbled Past',
       'Shot', 'Referee Ball-Drop', 'Shield', 'Interception', 'Half End',
       'Tactical Shift', 'Player Off', 'Player On', 'Substitution',
       '50/50'], dtype=object)

In [151]:
df.columns.to_list()

['index',
 'period',
 'timestamp',
 'minute',
 'second',
 'possession',
 'duration',
 'type_id',
 'type_name',
 'possession_team_id',
 'possession_team_name',
 'play_pattern_id',
 'play_pattern_name',
 'team_id',
 'team_name',
 'tactics_formation',
 'tactics_lineup',
 'shot_freeze_frame',
 'match_id',
 'obv_for_after',
 'obv_for_before',
 'obv_for_net',
 'obv_against_after',
 'obv_against_before',
 'obv_against_net',
 'obv_total_net',
 'player_id',
 'player_name',
 'position_id',
 'position_name',
 'pass_length',
 'pass_angle',
 'pass_pass_success_probability',
 'pass_recipient_id',
 'pass_recipient_name',
 'pass_height_id',
 'pass_height_name',
 'pass_body_part_id',
 'pass_body_part_name',
 'pass_type_id',
 'pass_type_name',
 'pass_pass_cluster_id',
 'pass_pass_cluster_label',
 'pass_pass_cluster_probability',
 'under_pressure',
 'pass_aerial_won',
 'pass_outcome_id',
 'pass_outcome_name',
 'duel_type_id',
 'duel_type_name',
 'clearance_right_foot',
 'clearance_body_part_id',
 'cleara

In [152]:
df_base  = df
dfxT = df_base.copy()
dfxT['type_name'] = dfxT['type_name'].astype(str)
dfxT = dfxT[(~dfxT['type_name'].str.contains('Corner'))]
dfxT = dfxT[(dfxT['type_name'].isin(['Pass', 'Carry'])) & (dfxT['pass_outcome_name']=="1")]
print(dfxT.shape)

xT = pd.read_csv('https://raw.githubusercontent.com/mckayjohns/youtube-videos/main/data/xT_Grid.csv', header=None)
#xT = pd.read_csv(r"C:\Users\lahd2\OneDrive\Mehdaf\Code\app\xT_Grid.csv", header=None)
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

dfxT['x1_bin_xT'] = pd.cut(dfxT['x'], bins=xT_cols, labels=False)
dfxT['y1_bin_xT'] = pd.cut(dfxT['y'], bins=xT_rows, labels=False)
dfxT['x2_bin_xT'] = pd.cut(
    np.where(dfxT['end_x'].notna(), dfxT['end_x'], dfxT['carry_end_x']),
    bins=xT_cols,
    labels=False
)

# Use np.where to choose between 'end_y' and 'carry_end_y'
dfxT['y2_bin_xT'] = pd.cut(
    np.where(dfxT['end_y'].notna(), dfxT['end_y'], dfxT['carry_end_y']),
    bins=xT_rows,
    labels=False
)
dfxT[['x1_bin_xT', 'y1_bin_xT', 'x2_bin_xT', 'y2_bin_xT']] = dfxT[
    ['x1_bin_xT', 'y1_bin_xT', 'x2_bin_xT', 'y2_bin_xT']
].fillna(0).astype(int)
dfxT['start_zone_value_xT'] = dfxT[['x1_bin_xT', 'y1_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)
dfxT['end_zone_value_xT'] = dfxT[['x2_bin_xT', 'y2_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)

dfxT['xT'] = dfxT['end_zone_value_xT'] - dfxT['start_zone_value_xT']
columns_to_drop = [
 'period',
 'timestamp',
 'minute',
 'second',
 'possession',
 'duration',
 'type_id',
 'type_name',
 'possession_team_id',
 'possession_team_name',
 'play_pattern_id',
 'play_pattern_name',
 'team_id',
 'team_name',
 'tactics_formation',
 'tactics_lineup',
 'shot_freeze_frame',
 'match_id',
 'obv_for_after',
 'obv_for_before',
 'obv_for_net',
 'obv_against_after',
 'obv_against_before',
 'obv_against_net',
 'obv_total_net',
 'player_id',
 'player_name',
 'position_id',
 'position_name',
 'pass_length',
 'pass_angle',
 'pass_pass_success_probability',
 'pass_recipient_id',
 'pass_recipient_name',
 'pass_height_id',
 'pass_height_name',
 'pass_body_part_id',
 'pass_body_part_name',
 'pass_type_id',
 'pass_type_name',
 'pass_pass_cluster_id',
 'pass_pass_cluster_label',
 'pass_pass_cluster_probability',
 'under_pressure',
 'pass_aerial_won',
 'pass_outcome_id',
 'pass_outcome_name',
 'duel_type_id',
 'duel_type_name',
 'clearance_right_foot',
 'clearance_body_part_id',
 'clearance_body_part_name',
 'clearance_head',
 'ball_receipt_outcome_id',
 'ball_receipt_outcome_name',
 'pass_switch',
 'out',
 'counterpress',
 'shot_statsbomb_xg',
 'shot_shot_execution_xg',
 'shot_shot_execution_xg_uplift',
 'shot_gk_positioning_xg_suppression',
 'shot_technique_id',
 'shot_technique_name',
 'shot_body_part_id',
 'shot_body_part_name',
 'shot_type_id',
 'shot_type_name',
 'shot_outcome_id',
 'shot_outcome_name',
 'goalkeeper_position_id',
 'goalkeeper_position_name',
 'goalkeeper_type_id',
 'goalkeeper_type_name',
 'clearance_aerial_won',
 'goalkeeper_body_part_id',
 'goalkeeper_body_part_name',
 'goalkeeper_outcome_id',
 'goalkeeper_outcome_name',
 'interception_outcome_id',
 'interception_outcome_name',
 'duel_outcome_id',
 'duel_outcome_name',
 'shot_first_time',
 'off_camera',
 'pass_assisted_shot_id',
 'pass_goal_assist',
 'pass_through_ball',
 'pass_technique_id',
 'pass_technique_name',
 'shot_one_on_one',
 'shot_gk_save_difficulty_xg',
 'shot_gk_shot_stopping_xg_suppression',
 'shot_key_pass_id',
 'goalkeeper_technique_id',
 'goalkeeper_technique_name',
 'dribble_outcome_id',
 'dribble_outcome_name',
 'ball_recovery_recovery_failure',
 'foul_committed_card_id',
 'foul_committed_card_name',
 'foul_won_defensive',
 'pass_shot_assist',
 'pass_inswinging',
 'pass_cross',
 'injury_stoppage_in_chain',
 'substitution_outcome_id',
 'substitution_outcome_name',
 'substitution_replacement_id',
 'substitution_replacement_name',
 'clearance_left_foot',
 'pass_no_touch',
 'shot_saved_off_target',
 'goalkeeper_shot_saved_off_target',
 '50_50_outcome_id',
 '50_50_outcome_name',
 'pass_cut_back',
 'block_offensive',
 'foul_committed_advantage',
 'foul_won_advantage',
 'pass_miscommunication',
 'dribble_nutmeg',
 'pass_deflected',
 'block_deflection',
 'pass_outswinging',
 'clearance_other',
 'bad_behaviour_card_id',
 'bad_behaviour_card_name',
 'foul_committed_type_id',
 'foul_committed_type_name',
 'dribble_overrun',
 'block_save_block',
 'miscontrol_aerial_won',
 'shot_aerial_won',
 'foul_committed_offensive',
 'pass_straight',
 'ball_recovery_offensive',
 'shot_deflected',
 'shot_open_goal',
 'goalkeeper_punched_out',
 'foul_committed_penalty',
 'foul_won_penalty',
 'shot_shot_shot_assist',
 'dribble_no_touch',
 'shot_saved_to_post',
 'goalkeeper_shot_saved_to_post',
 'shot_shot_goal_assist',
 'shot_follows_dribble',
 'goalkeeper_lost_out',
 'goalkeeper_lost_in_play',
 'player_off_permanent',
 'shot_redirect',
 'goalkeeper_success_out',
 'goalkeeper_success_in_play',
 'x',
 'y',
 'end_x',
 'end_y',
 'carry_end_x',
 'carry_end_y',
 'goalkeeper_end_x',
 'goalkeeper_end_y',
 'shot_end_x',
 'shot_end_y',
 'shot_end_z',
 'prog_pass',
 'prog_carry',
 'pass_or_carry_angle',
 'phase',
 'count_hi',
 'count_hsr',
 'count_high_acceleration',
 'count_high_deceleration',
 'count_medium_acceleration',
 'count_medium_deceleration',
 'count_sprint',
 'high_intensity_hi_distance',
 'high_speed_running_hsr_distance',
 'm/min',
 'max_speed',
 'running_distance',
 'sprinting_distance',
 'total_distance']
dfxT.drop(columns=columns_to_drop, inplace=True)
df = df.merge(dfxT, on='index', how='left')

(1313, 184)


C:\Users\lahd2\AppData\Local\Temp\ipykernel_39200\906181023.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfxT['start_zone_value_xT'] = dfxT[['x1_bin_xT', 'y1_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\lahd2\AppData\Local\Temp\ipykernel_39200\906181023.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfxT['end_zone_value_xT'] = dfxT[['x2_bin_xT', 'y2_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)


In [153]:

def find_shot_location(row, df):
    if row['type_name'] != 'Shot':
        return pd.Series([None, None, None])
    
    shot_index = row.name  # Get the row's index
    possession_id = row['possession']
    
    # Look for previous events in the same possession
    prev_events = df.loc[:shot_index - 1]
    prev_in_possession = prev_events[prev_events['possession'] == possession_id]

    # Look for the last Ball Receipt* event
    last_receipt = prev_in_possession[prev_in_possession['type_name'] == 'Ball Receipt*']
    
    if last_receipt.empty:
        return pd.Series([None, None, None])
    
    last_receipt_row = last_receipt.iloc[-1]
    x = last_receipt_row['x']
    y = last_receipt_row['y']
    
    in_box = (
        (x is not None and y is not None) and
        (
            # Right-side penalty box
            (102 <= x <= 120 and 18 <= y <= 62) or
            # Left-side penalty box
            (0 <= x <= 18 and 18 <= y <= 62)
        )
    )
    return pd.Series([x, y, in_box])
df[['Shoter_x', 'Shoter_y', 'in_box']] = df.apply(lambda row: find_shot_location(row, df), axis=1)


In [154]:
print(df[df['type_name'] == 'Shot'][['player_name', 'Shoter_x', 'Shoter_y', 'in_box','shot_type_name',]].head(10))


                                   player_name  Shoter_x  Shoter_y in_box  \
265                              Tokuma Suzuki       NaN       NaN   None   
337                               Isa Sakamoto     112.3      37.4   True   
350                               Mutsuki Kato     112.1      69.8  False   
523  Dawhan Fran Urano Da Purificação Oliveira       NaN       NaN   None   
577                          Tolgay Ali Arslan     100.2      37.3  False   
585                          Tolgay Ali Arslan       NaN       NaN   None   
591                            Pieros Soteriou       NaN       NaN   None   
624                                 Sho Sasaki     107.2      39.0   True   
681              Welton Felipe Paragua de Melo      69.2       6.1  False   
835              Welton Felipe Paragua de Melo      80.5       7.1  False   

    shot_type_name  
265      Free Kick  
337      Open Play  
350      Open Play  
523      Open Play  
577      Open Play  
585      Open Play  
591  

In [155]:
def players_passing_dfs(pname):
    # Filter for passes by the player
    dfpass = df[(df['type_name'] == 'Pass') & (df['player_name'] == pname)]
    acc_pass = dfpass[dfpass['pass_outcome_name'] == "1"]
    iac_pass = dfpass[dfpass['pass_outcome_name'] == 'Incomplete']
    
    # Calculate accurate pass percentage
    if len(dfpass) != 0:
        accurate_pass_perc = round((len(acc_pass) / len(dfpass)) * 100, 2)
    else:
        accurate_pass_perc = 0
    
    # Filter for progressive passes
    pro_pass = acc_pass[
        (acc_pass['prog_pass'] >= 9.25) & 
        (acc_pass['x'] >= 41) & 
        (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for through balls
    Thr_ball = dfpass[
        (dfpass['pass_technique_name'].str.contains('Throughball', case=False, na=False)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    Thr_ball_acc = Thr_ball[Thr_ball['pass_outcome_name'] == "1"]
    
    # Filter for long balls
    Lng_ball = dfpass[
        (dfpass['pass_pass_cluster_label'].str.contains('Long', case=False, na=False)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    Lng_ball_acc = Lng_ball[Lng_ball['pass_outcome_name'] =="1"]
    
    # Filter for crosses
    Crs_pass = dfpass[
        (dfpass['pass_cross'].astype(str).str.contains('TRUE', case=False, na=False)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    Crs_pass_acc = Crs_pass[Crs_pass['pass_outcome_name'] == "1"]
    
    # Filter for key passes
    key_pass = dfpass[dfpass['pass_shot_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
    
    # Filter for pre-assists
    df_no_carry = df[df['type_name'] != 'Carry'].reset_index(drop=True)
    pre_asst = df_no_carry[
        (df_no_carry['pass_goal_assist'].shift(-1).astype(str).str.contains('TRUE', case=False, na=False)) & 
        (df_no_carry['type_name'] == 'Pass') & 
        (df_no_carry['pass_outcome_name'] == "1") & 
        (df_no_carry['player_name'] == pname)
    ]
    
    # Filter for shot buildup
    shot_buildup = df_no_carry[
        (df_no_carry['pass_shot_assist'].shift(-1).astype(str).str.contains('TRUE', case=False, na=False)) & 
        (df_no_carry['type_name'] == 'Pass') & 
        (df_no_carry['pass_outcome_name'] == "1") & 
        (df_no_carry['player_name'] == pname)
    ]
    
    # Filter for goal assists
    g_assist = dfpass[dfpass['pass_goal_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
    
    # Filter for final third passes
    fnl_thd = acc_pass[
        (acc_pass['end_x'] >= 92.5) & 
        (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for penalty box passes
    pen_box = acc_pass[
        (acc_pass['end_x'] >= 102) & 
        (acc_pass['end_y'] >= 18.62) & 
        (acc_pass['end_y'] <= 62) & 
        (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for forward passes
    frwd_pass = dfpass[
        (dfpass['pass_or_carry_angle'] >= -85) & 
        (dfpass['pass_or_carry_angle'] <= 85) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for backward passes
    back_pass = dfpass[
        ((dfpass['pass_or_carry_angle'] >= 95) & (dfpass['pass_or_carry_angle'] <= 180)) | 
        ((dfpass['pass_or_carry_angle'] >= -180) & (dfpass['pass_or_carry_angle'] <= -95)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for sideways passes
    side_pass = dfpass[
        ((dfpass['pass_or_carry_angle'] >= 85) & (dfpass['pass_or_carry_angle'] <= 95)) | 
        ((dfpass['pass_or_carry_angle'] >= -95) & (dfpass['pass_or_carry_angle'] <= -85)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for accurate forward, backward, and sideways passes
    frwd_pass_acc = frwd_pass[frwd_pass['pass_outcome_name'] == "1"]
    back_pass_acc = back_pass[back_pass['pass_outcome_name'] == "1"]
    side_pass_acc = side_pass[side_pass['pass_outcome_name'] == "1"]
    
    # Filter for corners
    corners = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Corner', case=False, na=False)]
    corners_acc = corners[corners['pass_outcome_name'] == "1"]
    
    # Filter for free kicks
    freekik = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Goal Kick', case=False, na=False)]
    freekik_acc = freekik[freekik['pass_outcome_name'] == "1"]
    
    # Filter for throw-ins
    thins = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Throw In', case=False, na=False)]
    thins_acc = thins[thins['pass_outcome_name'] == "1"]
    
    # Filter for long balls
    lngball = dfpass[
        (dfpass['pass_pass_cluster_label'].astype(str).str.contains('Long', case=False, na=False)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    lngball_acc = lngball[lngball['pass_outcome_name'] == "1"]
    
    # Calculate forward pass accuracy
    if len(frwd_pass) != 0:
        Forward_Pass_Accuracy = round((len(frwd_pass_acc) / len(frwd_pass)) * 100, 2)
    else:
        Forward_Pass_Accuracy = 0
    
    # Calculate xT metrics
    df_xT_inc = (dfpass[dfpass['xT'] > 0])['xT'].sum().round(2)
    df_xT_dec = (dfpass[dfpass['xT'] < 0])['xT'].sum().round(2)
    total_xT = dfpass['xT'].sum().round(2)
    
    # Return the results as a dictionary
    return {
        'Name': pname,
        'Total_Passes': len(dfpass),
        'Accurate_Passes': len(acc_pass),
        'Miss_Pass': len(iac_pass),
        'Passing_Accuracy': accurate_pass_perc,
        'Progressive_Passes': len(pro_pass),
        'Chances_Created': len(key_pass),
        'Assists': len(g_assist),
        'Pre-Assists': len(pre_asst),
        'Buil-up_to_Shot': len(shot_buildup),
        'Final_Third_Passes': len(fnl_thd),
        'Passes_In_Penalty_Box': len(pen_box),
        'Through_Pass_Attempts': len(Thr_ball),
        'Accurate_Through_Passes': len(Thr_ball_acc),
        'Crosses_Attempts': len(Crs_pass),
        'Accurate_Crosses': len(Crs_pass_acc),
        'Longballs_Attempts': len(lngball),
        'Accurate_Longballs': len(lngball_acc),
        'Corners_Taken': len(corners),
        'Accurate_Corners': len(corners_acc),
        'Freekicks_Taken': len(freekik),
        'Accurate_Freekicks': len(freekik_acc),
        'ThrowIns_Taken': len(thins),
        'Accurate_ThrowIns': len(thins_acc),
        'Forward_Pass_Attempts': len(frwd_pass),
        'Accurate_Forward_Pass': len(frwd_pass_acc),
        'Side_Pass_Attempts': len(side_pass),
        'Accurate_Side_Pass': len(side_pass_acc),
        'Back_Pass_Attempts': len(back_pass),
        'Accurate_Back_Pass': len(back_pass_acc),
        'xT_Increased_From_Pass': df_xT_inc,
        'xT_Decreased_From_Pass': df_xT_dec,
        'Total_xT_From_Pass': total_xT 
    }


# Get unique player names
pnames = df['player_name'].unique()

# Create a list of dictionaries to store the counts for each player
data = []

for pname in pnames:
    counts = players_passing_dfs(pname)
    data.append(counts)

# Convert the list of dictionaries to a DataFrame
passing_dfs_df = pd.DataFrame(data)

# Sort the DataFrame by 'xT_Decreased_From_Pass' in descending order
passing_dfs_df = passing_dfs_df.sort_values(by='xT_Decreased_From_Pass', ascending=False).reset_index(drop=True)

# Display the DataFrame
passing_dfs_df

,Name,Total_Passes,Accurate_Passes,Miss_Pass,Passing_Accuracy,Progressive_Passes,Chances_Created,Assists,Pre-Assists,Buil-up_to_Shot,...,Accurate_ThrowIns,Forward_Pass_Attempts,Accurate_Forward_Pass,Side_Pass_Attempts,Accurate_Side_Pass,Back_Pass_Attempts,Accurate_Back_Pass,xT_Increased_From_Pass,xT_Decreased_From_Pass,Total_xT_From_Pass
0,NaN,0,0,0,0.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0.00,0.00,0.00
1,Sho Sasaki,45,37,6,82.22,7,0,0,0,0,...,12,31,27,3,3,5,5,0.09,-0.00,0.08
2,Issam Jebali,3,2,1,66.67,0,0,0,0,0,...,0,2,1,0,0,1,1,0.00,-0.00,-0.00
3,Takeru Kishimoto,1,1,0,100.00,0,0,0,0,0,...,0,1,1,0,0,0,0,0.00,0.00,0.00
4,Rin Mito,6,5,1,83.33,0,0,0,0,0,...,0,2,2,2,2,0,0,0.03,-0.00,0.03
5,Yotaro Nakajima,20,14,6,70.00,3,1,0,0,0,...,2,12,8,1,1,5,4,0.30,-0.00,0.30
6,Keisuke Osako,26,13,13,50.00,0,0,0,0,0,...,2,17,10,0,0,0,0,0.05,-0.00,0.04
7,Sota Koshimichi,1,0,1,0.00,0,0,0,0,0,...,0,1,0,0,0,0,0,0.00,0.00,0.00
8,Kota Yamada,14,9,3,64.29,1,0,0,0,0,...,0,4,1,1,1,7,6,0.04,-0.01,0.03
9,Shinnosuke Nakatani,44,34,9,77.27,1,0,0,0,0,...,2,29,24,2,2,7,7,0.10,-0.01,0.09


In [156]:
def players_passing_stats(pname):
     # Filter for passes by the player
    dfpass = df[(df['type_name'] == 'Pass') & (df['player_name'] == pname)]
    acc_pass = dfpass[dfpass['pass_outcome_name'] == "1"]
    iac_pass = dfpass[dfpass['pass_outcome_name'] == 'Incomplete']
    
    # Calculate accurate pass percentage
    if len(dfpass) != 0:
        accurate_pass_perc = round((len(acc_pass) / len(dfpass)) * 100, 2)
    else:
        accurate_pass_perc = 0
    
    # Filter for progressive passes
    pro_pass = acc_pass[
        (acc_pass['prog_pass'] >= 9.25) & 
        (acc_pass['x'] >= 41) & 
        (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for through balls
    Thr_ball = dfpass[
        (dfpass['pass_technique_name'].str.contains('Throughball', case=False, na=False)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    Thr_ball_acc = Thr_ball[Thr_ball['pass_outcome_name'] == "1"]
    
    # Filter for long balls
    Lng_ball = dfpass[
        (dfpass['pass_pass_cluster_label'].str.contains('Long', case=False, na=False)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    Lng_ball_acc = Lng_ball[Lng_ball['pass_outcome_name'] =="1"]
    
    # Filter for crosses
    Crs_pass = dfpass[
        (dfpass['pass_cross'].astype(str).str.contains('TRUE', case=False, na=False)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    Crs_pass_acc = Crs_pass[Crs_pass['pass_outcome_name'] == "1"]
    
    # Filter for key passes
    key_pass = dfpass[dfpass['pass_shot_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
    
    # Filter for pre-assists
    df_no_carry = df[df['type_name'] != 'Carry'].reset_index(drop=True)
    pre_asst = df_no_carry[
        (df_no_carry['pass_goal_assist'].shift(-1).astype(str).str.contains('TRUE', case=False, na=False)) & 
        (df_no_carry['type_name'] == 'Pass') & 
        (df_no_carry['pass_outcome_name'] == "1") & 
        (df_no_carry['player_name'] == pname)
    ]
    
    # Filter for shot buildup
    shot_buildup = df_no_carry[
        (df_no_carry['pass_shot_assist'].shift(-1).astype(str).str.contains('TRUE', case=False, na=False)) & 
        (df_no_carry['type_name'] == 'Pass') & 
        (df_no_carry['pass_outcome_name'] == "1") & 
        (df_no_carry['player_name'] == pname)
    ]
    
    # Filter for goal assists
    g_assist = dfpass[dfpass['pass_goal_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
    
    # Filter for final third passes
    fnl_thd = acc_pass[
        (acc_pass['end_x'] >= 92.5) & 
        (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for penalty box passes
    pen_box = acc_pass[
        (acc_pass['end_x'] >= 102) & 
        (acc_pass['end_y'] >= 18.62) & 
        (acc_pass['end_y'] <= 62) & 
        (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for forward passes
    frwd_pass = dfpass[
        (dfpass['pass_or_carry_angle'] >= -85) & 
        (dfpass['pass_or_carry_angle'] <= 85) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for backward passes
    back_pass = dfpass[
        ((dfpass['pass_or_carry_angle'] >= 95) & (dfpass['pass_or_carry_angle'] <= 180)) | 
        ((dfpass['pass_or_carry_angle'] >= -180) & (dfpass['pass_or_carry_angle'] <= -95)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for sideways passes
    side_pass = dfpass[
        ((dfpass['pass_or_carry_angle'] >= 85) & (dfpass['pass_or_carry_angle'] <= 95)) | 
        ((dfpass['pass_or_carry_angle'] >= -95) & (dfpass['pass_or_carry_angle'] <= -85)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    
    # Filter for accurate forward, backward, and sideways passes
    frwd_pass_acc = frwd_pass[frwd_pass['pass_outcome_name'] == "1"]
    back_pass_acc = back_pass[back_pass['pass_outcome_name'] == "1"]
    side_pass_acc = side_pass[side_pass['pass_outcome_name'] == "1"]
    
    # Filter for corners
    corners = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Corner', case=False, na=False)]
    corners_acc = corners[corners['pass_outcome_name'] == "1"]
    
    # Filter for free kicks
    freekik = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Goal Kick', case=False, na=False)]
    freekik_acc = freekik[freekik['pass_outcome_name'] == "1"]
    
    # Filter for throw-ins
    thins = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Throw In', case=False, na=False)]
    thins_acc = thins[thins['pass_outcome_name'] == "1"]
    
    # Filter for long balls
    lngball = dfpass[
        (dfpass['pass_pass_cluster_label'].astype(str).str.contains('Long', case=False, na=False)) & 
        (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    lngball_acc = lngball[lngball['pass_outcome_name'] == "1"]
    
    # Calculate forward pass accuracy
    if len(frwd_pass) != 0:
        Forward_Pass_Accuracy = round((len(frwd_pass_acc) / len(frwd_pass)) * 100, 2)
    else:
        Forward_Pass_Accuracy = 0
    
    # Calculate xT metrics
    df_xT_inc = (dfpass[dfpass['xT'] > 0])['xT'].sum().round(2)
    df_xT_dec = (dfpass[dfpass['xT'] < 0])['xT'].sum().round(2)
    total_xT = dfpass['xT'].sum().round(2)
    
    # Return the results as a dictionary
    return {
        'Name': pname,
        'Total_Passes': len(dfpass),
        'Accurate_Passes': len(acc_pass),
        'Miss_Pass': len(iac_pass),
        'Passing_Accuracy': accurate_pass_perc,
        'Progressive_Passes': len(pro_pass),
        'Chances_Created': len(key_pass),
        'Assists': len(g_assist),
        'Pre-Assists': len(pre_asst),
        'Buil-up_to_Shot': len(shot_buildup),
        'Final_Third_Passes': len(fnl_thd),
        'Passes_In_Penalty_Box': len(pen_box),
        'Through_Pass_Attempts': len(Thr_ball),
        'Accurate_Through_Passes': len(Thr_ball_acc),
        'Crosses_Attempts': len(Crs_pass),
        'Accurate_Crosses': len(Crs_pass_acc),
        'Longballs_Attempts': len(lngball),
        'Accurate_Longballs': len(lngball_acc),
        'Corners_Taken': len(corners),
        'Accurate_Corners': len(corners_acc),
        'Freekicks_Taken': len(freekik),
        'Accurate_Freekicks': len(freekik_acc),
        'ThrowIns_Taken': len(thins),
        'Accurate_ThrowIns': len(thins_acc),
        'Forward_Pass_Attempts': len(frwd_pass),
        'Accurate_Forward_Pass': len(frwd_pass_acc),
        'Side_Pass_Attempts': len(side_pass),
        'Accurate_Side_Pass': len(side_pass_acc),
        'Back_Pass_Attempts': len(back_pass),
        'Accurate_Back_Pass': len(back_pass_acc),
        'xT_Increased_From_Pass': df_xT_inc,
        'xT_Decreased_From_Pass': df_xT_dec,
        'Total_xT_From_Pass': total_xT 
    }


# Get unique player names
pnames = df['player_name'].unique()

# Create a list of dictionaries to store the counts for each player
data = []

for pname in pnames:
    counts = players_passing_stats(pname)
    data.append(counts)

# Convert the list of dictionaries to a DataFrame
passing_stats_df = pd.DataFrame(data)

# Sort the DataFrame by 'pr_count' in descending order
passing_stats_df = passing_stats_df.sort_values(by='xT_Decreased_From_Pass', ascending=False).reset_index(drop=True)
passing_stats_df
def player_carry_stats(pname):
    df_carry = df[(df['type_name']=='Carry') & (df['player_name']==pname)]
    led_shot1 = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['shot_outcome_name'].shift(-1)==('Saved'))]
    led_shot2 = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['shot_outcome_name'].shift(-1)==('Shot'))]
    led_shot = pd.concat([led_shot1, led_shot2])
    led_goal1 = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['pass_goal_assist'].shift(-1)==('TRUE'))]
    led_goal2 = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['shot_outcome_name'].shift(-1)=='Goal')]
    led_goal = pd.concat([led_goal1, led_goal2])
    pro_carry = df_carry[(df_carry['prog_carry']>=9.25) & (df_carry['x']>=42)]
    fth_carry = df_carry[(df_carry['x']<70) & (df_carry['carry_end_x']>=92.5)]
    box_entry = df_carry[(df_carry['carry_end_x']>=102) & (df_carry['carry_end_y']>=18) & (df_carry['carry_end_y']<=62) &
                 ~((df_carry['x']>=102) & (df_carry['y']>=18) & (df_carry['y']<=62))]
    disp = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['type_name'].shift(-1)=='Dispossessed')]
    df_to = df[(df['type_name']=='Dribble') & (df['player_name']==pname)]
    t_ons = df_to[df_to['dribble_outcome_name']=='Complete']
    t_onu = df_to[df_to['dribble_outcome_name']=='Incomplete']
    df_xT_inc = df_carry[df_carry['xT']>0]
    df_xT_dec = df_carry[df_carry['xT']<0]
    total_xT = df_carry['xT'].sum().round(2)
    df_carry = df_carry.copy()
    df_carry.loc[:, 'Length'] = np.sqrt((df_carry['x'] - df_carry['carry_end_x'])**2 + (df_carry['y'] - df_carry['carry_end_y'])**2)
    median_length = round(df_carry['Length'].median(),2)
    total_length = round(df_carry['Length'].sum(),2)
    if len(df_to)!=0:
        success_rate = round((len(t_ons)/len(df_to))*100, 2)
    else:
        success_rate = 0
    
    return {
        'player_name': pname,
        'Total_Carries': len(df_carry),
        'Progressive_Carries': len(pro_carry),
        'Carries_Led_to_Shot': len(led_shot),
        'Carries_Led_to_Goal': len(led_goal),
        'Carrier_Dispossessed': len(disp),
        'Carries_Into_Final_Third': len(fth_carry),
        'Carries_Into_Penalty_Box': len(box_entry),
        'Avg._Carry_Length': median_length,
        'Total_Carry_Length': total_length,
        'xT_Increased_From_Carries': df_xT_inc['xT'].sum().round(2),
        'xT_Decreased_From_Carries': df_xT_dec['xT'].sum().round(2),
        'Total_xT_From_Carries': total_xT,
        'TakeOn_Attempts': len(df_to),
        'Successful_TakeOns': len(t_ons)
    }

pnames = df['player_name'].unique()

# Create a list of dictionaries to store the counts for each player
data = []

for pname in pnames:
    counts = player_carry_stats(pname)
    data.append(counts)

# Convert the list of dictionaries to a DataFrame
carrying_stats_df = pd.DataFrame(data)

# Sort the DataFrame by 'pr_count' in descending order
carrying_stats_df = carrying_stats_df.sort_values(by='Total_Carries', ascending=False).reset_index(drop=True)
carrying_stats_df
def player_shooting_stats(pname):
    # goal = df[(df['player_name']==pname) & (df['type_name']=='Goal') & (~df['type_name'].str.contains('BigChance'))]
    # miss = df[(df['player_name']==pname) & (df['type_name']=='MissedShots') & (~df['type_name'].str.contains('BigChance'))]
    # save = df[(df['player_name']==pname) & (df['type_name']=='SavedShot') & (~df['type_name'].str.contains('BigChance'))]
    # post = df[(df['player_name']==pname) & (df['type_name']=='ShotOnPost') & (~df['type_name'].str.contains('BigChance'))]
    shots_df=df

    op_sh = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_type_name'].str.contains('Open Play'))]

    goal = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Goal') & 
                     (shots_df['type_name']!='Own Goal For')]
    miss = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Off T')]
    save = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Saved')]
    blok = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Blocked')]
    post = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Post')]


    shots = df[(df['player_name']==pname) & 
           ((df['shot_outcome_name']=='Goal') | 
            (df['shot_outcome_name']=='Off T') | 
            (df['shot_outcome_name']=='Saved') | 
            (df['shot_outcome_name']=='Post') | 
            (df['shot_outcome_name']=='Blocked'))]
    shots = shots.copy()
    shots.loc[:, 'Length'] = np.sqrt((shots['Shoter_x'] - 120)**2 + (shots['Shoter_y'] - 40)**2)
    avg_dist = round(shots['Length'].mean(), 2)
    xG = shots_df[(shots_df['player_name']==pname)]['shot_statsbomb_xg'].sum().round(2)
    xGOT = shots_df[(shots_df['player_name']==pname)]['shot_gk_save_difficulty_xg'].sum().round(2)
    in_box = shots[shots['in_box'] == True]


    return {
        'player_name': pname, 
        'Total_Shots': len(shots),
        'Open_Play_Shots': len(op_sh),
        'Goals': len(goal),
        'Shot_On_Post': len(post),
        'Shot_On_Target': len(save),
        'Shot_Off_Target': len(miss),
        'Shot_Blocked': len(blok),
        'Shots_From_Inside_The_Box': len(in_box),
        'Shots_From_Outside_The_Box': len(shots) - len(in_box),
        'Avg._Shot_Distance': avg_dist,
        'xG': xG,
        'xGOT': xGOT
    }

pnames = df['player_name'].unique()

# Create a list of dictionaries to store the counts for each player
data = []

for pname in pnames:
    counts = player_shooting_stats(pname)
    data.append(counts)

# Convert the list of dictionaries to a DataFrame
shooting_stats_df = pd.DataFrame(data)

# Sort the DataFrame by 'pr_count' in descending order
shooting_stats_df = shooting_stats_df.sort_values(by='Total_Shots', ascending=False).reset_index(drop=True)
shooting_stats_df

,player_name,Total_Shots,Open_Play_Shots,Goals,Shot_On_Post,Shot_On_Target,Shot_Off_Target,Shot_Blocked,Shots_From_Inside_The_Box,Shots_From_Outside_The_Box,Avg._Shot_Distance,xG,xGOT
0,Sho Sasaki,4,4,0,0,1,1,2,4,0,8.80,0.43,0.62
1,Tolgay Ali Arslan,3,3,0,0,1,1,1,1,2,18.37,0.23,0.31
2,Shunki Higashi,2,1,0,0,0,1,1,0,2,26.53,0.08,0.00
3,Isa Sakamoto,2,2,2,0,0,0,0,2,0,12.25,0.83,1.63
4,Welton Felipe Paragua de Melo,2,2,0,0,2,0,0,0,2,56.24,0.14,0.24
5,Tsukasa Shiotani,2,3,0,0,0,2,0,0,2,25.68,0.07,0.00
6,Mutsuki Kato,2,2,1,0,0,0,1,1,1,18.92,0.45,0.74
7,Hayao Kawabe,1,1,0,0,0,0,1,1,0,7.02,0.03,0.00
8,Ryoya Yamashita,1,1,0,0,0,1,0,0,1,28.32,0.09,0.00
9,Keisuke Kurokawa,1,0,0,0,1,0,0,0,1,NaN,0.05,0.26


In [157]:
def player_shooting_stats(pname):
    # goal = df[(df['player_name']==pname) & (df['type_name_name']=='Goal') & (~df['type_name_name'].str.contains('BigChance'))]
    # miss = df[(df['player_name']==pname) & (df['type_name_name']=='MissedShots') & (~df['type_name_name'].str.contains('BigChance'))]
    # save = df[(df['player_name']==pname) & (df['type_name_name']=='SavedShot') & (~df['type_name_name'].str.contains('BigChance'))]
    # post = df[(df['player_name']==pname) & (df['type_name_name']=='ShotOnPost') & (~df['type_name_name'].str.contains('BigChance'))]
    shots_df=df

    op_sh = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_type_name'].str.contains('Open Play'))]

    goal = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Goal') & 
                     (shots_df['type_name']!='Own Goal For')]
    miss = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Off T')]
    save = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Saved')]
    blok = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Blocked')]
    post = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Post')]


    shots = df[(df['player_name']==pname) & 
           ((df['shot_outcome_name']=='Goal') | 
            (df['shot_outcome_name']=='Off T') | 
            (df['shot_outcome_name']=='Saved') | 
            (df['shot_outcome_name']=='Post') | 
            (df['shot_outcome_name']=='Blocked'))]
    shots = shots.copy()
    shots.loc[:, 'Length'] = np.sqrt((shots['Shoter_x'] - 120)**2 + (shots['Shoter_y'] - 40)**2)
    avg_dist = round(shots['Length'].mean(), 2)
    xG = shots_df[(shots_df['player_name']==pname)]['shot_statsbomb_xg'].sum().round(2)
    xGOT = shots_df[(shots_df['player_name']==pname)]['shot_gk_save_difficulty_xg'].sum().round(2)
    in_box = shots[shots['in_box'] == True]


    return {
        'player_name': pname, 
        'Total_Shots': len(shots),
        'Open_Play_Shots': len(op_sh),
        'Goals': len(goal),
        'Shot_On_Post': len(post),
        'Shot_On_Target': len(save),
        'Shot_Off_Target': len(miss),
        'Shot_Blocked': len(blok),
        'Shots_From_Inside_The_Box': len(in_box),
        'Shots_From_Outside_The_Box': len(shots) - len(in_box),
        'Avg._Shot_Distance': avg_dist,
        'xG': xG,
        'xGOT': xGOT
    }

pnames = df['player_name'].unique()

# Create a list of dictionaries to store the counts for each player
data = []

for pname in pnames:
    counts = player_shooting_stats(pname)
    data.append(counts)

# Convert the list of dictionaries to a DataFrame
shooting_stats_df = pd.DataFrame(data)

# Sort the DataFrame by 'pr_count' in descending order
shooting_stats_df = shooting_stats_df.sort_values(by='Total_Shots', ascending=False).reset_index(drop=True)
shooting_stats_df
def player_pass_receiving_stats(pname):
    # Step 1: Get all successful passes
    successful_passes = df[(df['type_name'] == 'Pass') & (df['pass_outcome_name'] == '1')]

    # Step 2: Identify passes where the next row is received by `pname`
    pass_indexes = successful_passes.index
    next_rows = df.loc[pass_indexes + 1]
    received_passes = successful_passes[next_rows['player_name'].values == pname]
    
    # This is the reception index (row after the pass)
    reception_indexes = received_passes.index + 1
    receptions = df.loc[reception_indexes].copy()
    receptions['x'] = received_passes['end_x'].values
    receptions['y'] = received_passes['end_y'].values

    # Step 3: Track receiver's next action
    next_action_indexes = reception_indexes + 1
    next_actions = df.loc[next_action_indexes].copy()

    # Step 4: Ball retention = not losing the ball in next action
    ball_retain = next_actions[
        (next_actions['type_name'] != 'Foul Committed') &
        (next_actions['type_name'] != 'Dispossessed')
    ]

    if len(next_actions) != 0:
        ball_retention = round((len(ball_retain) / len(next_actions)) * 100, 2)
    else:
        ball_retention = 0

    # Step 5: Now apply filters using the pass that led to these receptions
    dfp = received_passes.copy()

    dfkp = dfp[dfp['pass_shot_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
    dfas = dfp[dfp['pass_goal_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
    dfnt = dfp[dfp['end_x'] >= 92]
    dfpen = dfp[(dfp['end_x'] >= 102) & (dfp['end_y'] >= 18) & (dfp['end_y'] <= 62)]
    dfpro = dfp[
        (dfp['prog_pass'] >= 9.25) &
        (dfp['x'] >= 41) &
        (~dfp['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    dfcros = dfp[
        (dfp['pass_cross'].astype(str).str.contains('TRUE', case=False, na=False)) &
        (~dfp['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    dfxT = dfp[dfp['xT'] >= 0]
    dflb = dfp[
        (dfp['pass_pass_cluster_label'].astype(str).str.contains('Long', case=False, na=False)) &
        (~dfp['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
    ]
    cutback = dfp[
        ((dfp['x'] >= 100.5) & (dfp['x'] <= 120) &
         (((dfp['y'] >= 46.6) & (dfp['y'] <= 64)) | ((dfp['y'] >= 16) & (dfp['y'] <= 32))) &
         (dfp['end_y'] >= 32) & (dfp['end_y'] <= 46.5) & (dfp['end_x'] >= 93.3))
    ]

    # Most passes received from the same player
    if len(dfp) > 0:
        top_sender = dfp['player_name'].value_counts().idxmax()
        top_sender_count = dfp['player_name'].value_counts().max()
        most_from = f'{top_sender}({top_sender_count})'
    else:
        most_from = 'None(0)'

    return {
        'Name': pname,
        'Total_Passes_Received': len(dfp),
        'Key_Passes_Received': len(dfkp),
        'Assists_Received': len(dfas),
        'Progressive_Passes_Received': len(dfpro),
        'Passes_Received_In_Final_Third': len(dfnt),
        'Passes_Received_In_Opponent_Penalty_Box': len(dfpen),
        'Crosses_Received': len(dfcros),
        'Longballs_Received': len(dflb),
        'Cutbacks_Received': len(cutback),
        'Ball_Retention': ball_retention,
        'xT_Received': round(dfxT['xT'].sum(), 2),
        'Avg._Distance_Of_Pass_Receiving_Form_Oppo._Goal_Line': round(120 - dfp['end_x'].median(), 2) if not dfp.empty else None,
        'Most_Passes_Received_From_(Count)': most_from
 
    }

pnames = df['player_name'].unique()

# Create a list of dictionaries to store the counts for each player
data = []

for pname in pnames:
    counts = player_pass_receiving_stats(pname)
    data.append(counts)

# Convert the list of dictionaries to a DataFrame
player_pass_receiving_stats_df = pd.DataFrame(data)

# Sort the DataFrame by 'pr_count' in descending order
player_pass_receiving_stats_df = player_pass_receiving_stats_df.sort_values(by='Total_Passes_Received', ascending=False).reset_index(drop=True)
player_pass_receiving_stats_df

,Name,Total_Passes_Received,Key_Passes_Received,Assists_Received,Progressive_Passes_Received,Passes_Received_In_Final_Third,Passes_Received_In_Opponent_Penalty_Box,Crosses_Received,Longballs_Received,Cutbacks_Received,Ball_Retention,xT_Received,Avg._Distance_Of_Pass_Receiving_Form_Oppo._Goal_Line,Most_Passes_Received_From_(Count)
0,Hayato Araki,39,0,0,3,2,1,0,5,0,100.0,0.09,74.10,Tsukasa Shiotani(13)
1,Keisuke Kurokawa,37,0,0,4,11,1,0,0,0,100.0,0.10,43.90,Shota Fukuoka(6)
2,Tolgay Ali Arslan,37,2,0,8,6,1,0,6,0,100.0,0.19,44.80,Shunki Higashi(10)
3,Tsukasa Shiotani,33,0,0,1,2,0,0,2,0,100.0,0.03,74.50,Hayato Araki(11)
4,Tokuma Suzuki,33,1,0,3,4,0,0,0,0,100.0,0.07,51.90,Keisuke Kurokawa(8)
5,Sho Sasaki,32,2,0,3,5,2,0,3,0,100.0,0.51,59.55,Hayato Araki(12)
6,Shinnosuke Nakatani,32,0,0,4,2,2,2,0,0,100.0,0.17,81.70,Dawhan Fran Urano Da Purificação Oliveira(9)
7,Shota Fukuoka,32,0,0,4,2,0,0,1,0,100.0,0.06,85.90,Keisuke Kurokawa(7)
8,Dawhan Fran Urano Da Purificação Oliveira,30,0,0,4,7,2,1,0,0,100.0,0.29,59.75,Tokuma Suzuki(7)
9,Shuto Nakano,29,0,0,5,9,2,0,5,0,100.0,0.17,43.50,Tsukasa Shiotani(10)


In [158]:
def player_defensive_stats(pname):

    playerdf = df[df["player_name"] == pname]

    ball_wins = playerdf[
        (playerdf["type_name"] == 'Interception') | (playerdf["type_name"] == 'Ball Recovery')
    ]

    f_third = ball_wins[ball_wins['x'] >= 80]
    m_third = ball_wins[(ball_wins['x'] > 40) & (ball_wins['x'] < 80)]
    d_third = ball_wins[ball_wins['x'] <= 40]

    tk = playerdf[
        (playerdf["type_name"] == 'Duel') &
        (playerdf['duel_type_name'].str.contains('Tackle', case=False, na=False))
    ]

    tk_u = playerdf[
        (playerdf["type_name"] == 'Duel') &
        (playerdf['duel_type_name'].str.contains('Tackle', case=False, na=False)) &
        (playerdf['duel_outcome_name'].isin(['Lost In Play', 'Lost']))
    ]

    intc = df[
    (df['player_name'] == pname) &
    (df['type_name'] == 'Interception') &
    (df['interception_outcome_name'].isin(['Success In Play', 'Won']))
]


    br = df[
    (df['player_name'] == pname) &
    (df['type_name'] == 'Ball Recovery') &
    (df['ball_recovery_recovery_failure'].astype(str) != 'True')
]


    cl = playerdf[playerdf["type_name"] == 'Clearance']
    fl = playerdf[playerdf["type_name"] == 'Foul Committed']

    ar = playerdf[
        (playerdf["type_name"] == 'Duel') &
        (playerdf['duel_type_name'].str.contains('Aerial', case=False, na=False))
    ]

    ar_u = playerdf[
        (playerdf["type_name"] == 'Duel') &
        (playerdf['duel_type_name'].str.contains('Aerial', case=False, na=False)) &
        (playerdf['duel_outcome_name'] == 'Aerial Lost')
    ]

    pass_bl = playerdf[playerdf["type_name"] == 'Block']

    shot_bl = playerdf[
        (playerdf["type_name"] == 'Block') &
        (playerdf['block_save_block'].astype(str).str.contains('TRUE', case=False, na=False))
    ]

    drb_pst = playerdf[playerdf["type_name"] == 'Dribbled Past']

    drb_tkl = df[
        (df["player_name"] == pname) &
        (df["type_name"] == 'Duel') &
        (df['duel_type_name'].str.contains('Tackle', case=False, na=False)) &
        (df["duel_outcome_name"].isin(['Lost In Play', 'Lost'])) &
        (df["type_name"].shift(1) == 'Dribble')
    ]
    # 1. Identify all defensive failures by the player
    errors = df[
        (df['player_name'] == pname) & (
            # Unsuccessful tackle or duel
            ((df['type_name'] == 'Duel') &
            (df['duel_type_name'].str.contains('Tackle|Aerial', case=False, na=False)) &
            (df['duel_outcome_name'].isin(['Lost In Play', 'Lost', 'Aerial Lost']))) |

            # Interception attempt that doesn't stop the ball (can add more logic if needed)

            # Fouls committed
            (df['type_name'] == 'Foul Committed')
        )
    ]

    # 2. Initialize counters
    err_lat = 0  # Error leading to shot
    err_lgl = 0  # Error leading to goal

    # 3. Loop through those events and check the next 1-2 events
    for idx in errors.index:
        next_events = df.loc[idx + 1: idx + 2]
        if not next_events.empty and 'type_name' in next_events:
            shot_events = next_events[next_events['type_name'] == 'Shot']
            if not shot_events.empty:
                err_lat += 1
                if 'shot_outcome_name' in shot_events and any(shot_events['shot_outcome_name'] == 'Goal'):
                    err_lgl += 1
    return {
        'Player': pname,
        'Interceptions': len(intc),
        'Ball Recoveries': len(br),
        'Tackles': len(tk),
        'Unsuccessful Tackles': len(tk_u),
        'Clearances': len(cl),
        'Fouls Committed': len(fl),
        'Aerial Duels': len(ar),
        'Aerial Duels Lost': len(ar_u),
        'Passes Blocked': len(pass_bl),
        'Shots Blocked': len(shot_bl),
        'Dribbled Past': len(drb_pst),
        'Dribble Tackles': len(drb_tkl),
        'Ball Wins in Final Third': len(f_third),
        'Ball Wins in Middle Third': len(m_third),
        'Ball Wins in Defensive Third': len(d_third),
        'Errors Leading to Shot': err_lat,
        'Errors Leading to Goal': err_lgl
    }
names = df['player_name'].unique()

# Create a list of dictionaries to store the counts for each player
data = []

for pname in pnames:
    counts = player_defensive_stats(pname)
    data.append(counts)

# Convert the list of dictionaries to a DataFrame
player_defensive_stats = pd.DataFrame(data)
player_defensive_stats

,Player,Interceptions,Ball Recoveries,Tackles,Unsuccessful Tackles,Clearances,Fouls Committed,Aerial Duels,Aerial Duels Lost,Passes Blocked,Shots Blocked,Dribbled Past,Dribble Tackles,Ball Wins in Final Third,Ball Wins in Middle Third,Ball Wins in Defensive Third,Errors Leading to Shot,Errors Leading to Goal
0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Isa Sakamoto,0,3,0,0,1,2,2,0,1,0,0,0,1,1,1,0,0
2,Shota Fukuoka,2,1,4,0,9,0,2,0,3,0,0,0,0,3,3,0,0
3,Shuto Nakano,1,3,2,1,0,1,3,0,2,0,1,0,4,1,1,0,0
4,Dawhan Fran Urano Da Purificação Oliveira,0,6,5,0,6,1,1,0,3,0,0,0,1,3,2,0,0
5,Welton Felipe Paragua de Melo,0,4,1,0,0,3,6,0,2,0,0,0,2,1,1,0,0
6,Tsukasa Shiotani,1,9,5,0,0,0,0,0,0,0,3,0,4,5,2,0,0
7,Taishi Matsumoto,2,5,4,2,2,2,1,0,2,0,1,1,2,5,0,0,0
8,Tokuma Suzuki,0,5,2,0,2,1,2,0,3,0,3,0,2,3,2,0,0
9,Shinnosuke Nakatani,0,4,4,1,5,1,1,0,1,0,0,1,1,2,3,0,0


In [159]:
def touches_stats(pname):
    touch_types = [
        'Ball Receipt*',"Pass"
    ]
    df_player = df[df['player_name'] == pname]
    touches = df_player[df_player['type_name'].isin(touch_types)]
    final_third = touches[touches['x'] >= 80]
    pen_box = touches[(touches['x'] >= 102) & (touches['y'] >= 18) & (touches['y'] <= 62)]


    return {
        'Name': pname,
        'Total_Touches': len(touches),
        'Touches_In_Final_Third': len(final_third),
        'Touches_In_Opponent_Penalty_Box': len(pen_box),


    }
    

# Create a list of dictionaries to store the counts for each player
data = []

for pname in pnames:
    counts = touches_stats(pname)
    data.append(counts)

# Convert the list of dictionaries to a DataFrame
touches_stats_df = pd.DataFrame(data)

# Sort the DataFrame by 'pr_count' in descending order

touches_stats_df = touches_stats_df.sort_values(by='Total_Touches', ascending=False).reset_index(drop=True)
touches_stats_df
# Rename the column in shooting_stats_df to match passing_stats_df
# Rename columns in all DataFrames to ensure consistency
# Rename columns in all DataFrames to ensure consistency
shooting_stats_df = shooting_stats_df.rename(columns={'player_name': 'Name'})
passing_stats_df = passing_stats_df.rename(columns={'Name': 'Name'})  # Ensure 'Name' is consistent
carrying_stats_df = carrying_stats_df.rename(columns={'player_name': 'Name'})
player_pass_receiving_stats_df = player_pass_receiving_stats_df.rename(columns={'player_name': 'Name'})
player_defensive_stats_df = player_defensive_stats_df.rename(columns={'player_name': 'Name'})
touches_stats_df = touches_stats_df.rename(columns={'player_name': 'Name'})  # Rename if necessary

# Sort touches_stats_df by 'Total_Touches' (if not already sorted)
touches_stats_df = touches_stats_df.sort_values(by='Total_Touches', ascending=False).reset_index(drop=True)

# Merge all DataFrames
player_stats_df = shooting_stats_df.merge(passing_stats_df, on='Name', how='left')
player_stats_df = player_stats_df.merge(carrying_stats_df, on='Name', how='left')
player_stats_df = player_stats_df.merge(player_pass_receiving_stats_df, on='Name', how='left')
player_stats_df = player_stats_df.merge(player_defensive_stats_df, on='Name', how='left')
player_stats_df = player_stats_df.merge(touches_stats_df, on='Name', how='left')

# Display the final DataFrame
# 
player_stats_df

,Name,Total_Shots,Open_Play_Shots,Goals,Shot_On_Post,Shot_On_Target,Shot_Off_Target,Shot_Blocked,Shots_From_Inside_The_Box,Shots_From_Outside_The_Box,...,Fouls_Committed,Fouls_Infront_Of_Own_Penalty_Box,Error_Led_to_Shot,Error_Led_to_Goal,Possession Win in Final third,Possession Win in Mid third,Possession Win in Defensive third,Total_Touches,Touches_In_Final_Third,Touches_In_Opponent_Penalty_Box
0,Sho Sasaki,4,4,0,0,1,1,2,4,0,...,1,0,0,0,3,0,1,73,4,2
1,Tolgay Ali Arslan,3,3,0,0,1,1,1,1,2,...,1,1,0,0,3,3,2,78,28,4
2,Shunki Higashi,2,1,0,0,0,1,1,0,2,...,1,0,0,0,2,2,2,83,49,3
3,Isa Sakamoto,2,2,2,0,0,0,0,2,0,...,2,0,0,0,1,1,1,47,15,2
4,Welton Felipe Paragua de Melo,2,2,0,0,2,0,0,0,2,...,3,0,0,0,2,1,1,64,27,4
5,Tsukasa Shiotani,2,3,0,0,0,2,0,0,2,...,0,0,0,0,4,5,2,92,10,1
6,Mutsuki Kato,2,2,1,0,0,0,1,1,1,...,2,0,0,0,0,2,0,73,36,6
7,Hayao Kawabe,1,1,0,0,0,0,1,1,0,...,4,0,0,0,2,1,1,32,11,0
8,Ryoya Yamashita,1,1,0,0,0,1,0,0,1,...,1,0,0,0,3,2,1,48,16,3
9,Keisuke Kurokawa,1,0,0,0,1,0,0,0,1,...,2,0,0,0,0,3,5,78,23,2


In [160]:
# Calculate total OBV contributions for each player
player_obv_df = (
    df.groupby('player_name')[['obv_for_net', 'obv_against_net']]
    .sum()
    .reset_index()
    .rename(columns={'player_name': 'Name'})
)

# Optional: Add a combined OBV net value (offensive - defensive)
player_obv_df['obv_total_net'] = player_obv_df['obv_for_net'] - player_obv_df['obv_against_net']


In [163]:
player_stats_df = player_stats_df.merge(player_obv_df, on='Name', how='left')
player_stats_df

,Name,Total_Shots,Open_Play_Shots,Goals,Shot_On_Post,Shot_On_Target,Shot_Off_Target,Shot_Blocked,Shots_From_Inside_The_Box,Shots_From_Outside_The_Box,...,Possession Win in Defensive third,Total_Touches,Touches_In_Final_Third,Touches_In_Opponent_Penalty_Box,obv_for_net_x,obv_against_net_x,obv_total_net_x,obv_for_net_y,obv_against_net_y,obv_total_net_y
0,Pieros Soteriou,5,4,0,0,3,2,0,2,3,...,1.0,50,31,6,-1.118600,0.025100,-1.143700,-1.118600,0.025100,-1.143700
1,Yuki Ohashi,5,7,2,0,1,2,0,3,2,...,NaN,83,55,8,0.879500,-0.023306,0.902806,0.879500,-0.023306,0.902806
2,Shinzo Koroki,3,3,0,0,0,1,2,3,0,...,NaN,20,15,5,-0.268600,0.016700,-0.285300,-0.268600,0.016700,-0.285300
3,Hayato Araki,3,3,0,0,0,1,2,3,0,...,3.0,67,2,2,-0.097800,-0.147809,0.050009,-0.097800,-0.147809,0.050009
4,Takumu Kawamura,2,2,0,0,2,0,0,0,2,...,NaN,54,9,1,-0.004402,0.023000,-0.027402,-0.004402,0.023000,-0.027402
5,Mutsuki Kato,2,2,0,0,1,1,0,0,2,...,0.0,62,35,7,-0.107500,-0.056100,-0.051400,-0.107500,-0.056100,-0.051400
6,Yoshio Koizumi,2,3,0,0,1,0,1,1,1,...,NaN,90,30,5,0.024900,0.615801,-0.590901,0.024900,0.615801,-0.590901
7,Makoto Mitsuta,2,2,0,0,0,1,1,0,2,...,0.0,129,44,2,0.616694,-0.046994,0.663688,0.616694,-0.046994,0.663688
8,Takahiro Sekine,2,2,0,0,1,0,1,2,0,...,NaN,52,19,4,0.168396,-0.099409,0.267805,0.168396,-0.099409,0.267805
9,Shunki Higashi,2,1,0,0,0,1,1,0,2,...,2.0,98,43,3,0.542300,-0.091200,0.633500,0.542300,-0.091200,0.633500


In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mplsoccer import Pitch
import os
import ast
from unidecode import unidecode
from scipy.spatial import ConvexHull
import os
import pandas as pd
from glob import glob

# 📁 Path to your folder of match files
folder_path = r"E:\Ai_com\match_enriched"
match_files = glob(os.path.join(folder_path, "match_*.csv"))

# 📦 Loop through all matches
for file_path in match_files:
    match_id = os.path.splitext(os.path.basename(file_path))[0].split("_")[1]
    df = pd.read_csv(file_path)
    df_base  = df
    dfxT = df_base.copy()
    dfxT['type_name'] = dfxT['type_name'].astype(str)
    dfxT = dfxT[(~dfxT['type_name'].str.contains('Corner'))]
    dfxT = dfxT[(dfxT['type_name'].isin(['Pass', 'Carry'])) & (dfxT['pass_outcome_name']=="1")]
    print(dfxT.shape)

    xT = pd.read_csv('https://raw.githubusercontent.com/mckayjohns/youtube-videos/main/data/xT_Grid.csv', header=None)
    #xT = pd.read_csv(r"C:\Users\lahd2\OneDrive\Mehdaf\Code\app\xT_Grid.csv", header=None)
    xT = np.array(xT)
    xT_rows, xT_cols = xT.shape

    dfxT['x1_bin_xT'] = pd.cut(dfxT['x'], bins=xT_cols, labels=False)
    dfxT['y1_bin_xT'] = pd.cut(dfxT['y'], bins=xT_rows, labels=False)
    dfxT['x2_bin_xT'] = pd.cut(
        np.where(dfxT['end_x'].notna(), dfxT['end_x'], dfxT['carry_end_x']),
        bins=xT_cols,
        labels=False
    )

    # Use np.where to choose between 'end_y' and 'carry_end_y'
    dfxT['y2_bin_xT'] = pd.cut(
        np.where(dfxT['end_y'].notna(), dfxT['end_y'], dfxT['carry_end_y']),
        bins=xT_rows,
        labels=False
    )
    dfxT[['x1_bin_xT', 'y1_bin_xT', 'x2_bin_xT', 'y2_bin_xT']] = dfxT[
        ['x1_bin_xT', 'y1_bin_xT', 'x2_bin_xT', 'y2_bin_xT']
    ].fillna(0).astype(int)
    dfxT['start_zone_value_xT'] = dfxT[['x1_bin_xT', 'y1_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)
    dfxT['end_zone_value_xT'] = dfxT[['x2_bin_xT', 'y2_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)

    dfxT['xT'] = dfxT['end_zone_value_xT'] - dfxT['start_zone_value_xT']
    columns_to_drop = [
    'period',
    'timestamp',
    'minute',
    'second',
    'possession',
    'duration',
    'type_id',
    'type_name',
    'possession_team_id',
    'possession_team_name',
    'play_pattern_id',
    'play_pattern_name',
    'team_id',
    'team_name',
    'tactics_formation',
    'tactics_lineup',
    'shot_freeze_frame',
    'match_id',
    'obv_for_after',
    'obv_for_before',
    'obv_for_net',
    'obv_against_after',
    'obv_against_before',
    'obv_against_net',
    'obv_total_net',
    'player_id',
    'player_name',
    'position_id',
    'position_name',
    'pass_length',
    'pass_angle',
    'pass_pass_success_probability',
    'pass_recipient_id',
    'pass_recipient_name',
    'pass_height_id',
    'pass_height_name',
    'pass_body_part_id',
    'pass_body_part_name',
    'pass_type_id',
    'pass_type_name',
    'pass_pass_cluster_id',
    'pass_pass_cluster_label',
    'pass_pass_cluster_probability',
    'under_pressure',
    'pass_aerial_won',
    'pass_outcome_id',
    'pass_outcome_name',
    'duel_type_id',
    'duel_type_name',
    'clearance_right_foot',
    'clearance_body_part_id',
    'clearance_body_part_name',
    'clearance_head',
    'ball_receipt_outcome_id',
    'ball_receipt_outcome_name',
    'pass_switch',
    'out',
    'counterpress',
    'shot_statsbomb_xg',
    'shot_shot_execution_xg',
    'shot_shot_execution_xg_uplift',
    'shot_gk_positioning_xg_suppression',
    'shot_technique_id',
    'shot_technique_name',
    'shot_body_part_id',
    'shot_body_part_name',
    'shot_type_id',
    'shot_type_name',
    'shot_outcome_id',
    'shot_outcome_name',
    'goalkeeper_position_id',
    'goalkeeper_position_name',
    'goalkeeper_type_id',
    'goalkeeper_type_name',
    'clearance_aerial_won',
    'goalkeeper_body_part_id',
    'goalkeeper_body_part_name',
    'goalkeeper_outcome_id',
    'goalkeeper_outcome_name',
    'interception_outcome_id',
    'interception_outcome_name',
    'duel_outcome_id',
    'duel_outcome_name',
    'shot_first_time',
    'off_camera',
    'pass_assisted_shot_id',
    'pass_goal_assist',
    'pass_through_ball',
    'pass_technique_id',
    'pass_technique_name',
    'shot_one_on_one',
    'shot_gk_save_difficulty_xg',
    'shot_gk_shot_stopping_xg_suppression',
    'shot_key_pass_id',
    'goalkeeper_technique_id',
    'goalkeeper_technique_name',
    'dribble_outcome_id',
    'dribble_outcome_name',
    'ball_recovery_recovery_failure',
    'foul_committed_card_id',
    'foul_committed_card_name',
    'foul_won_defensive',
    'pass_shot_assist',
    'pass_inswinging',
    'pass_cross',
    'injury_stoppage_in_chain',
    'substitution_outcome_id',
    'substitution_outcome_name',
    'substitution_replacement_id',
    'substitution_replacement_name',
    'clearance_left_foot',
    'pass_no_touch',
    'shot_saved_off_target',
    'goalkeeper_shot_saved_off_target',
    '50_50_outcome_id',
    '50_50_outcome_name',
    'pass_cut_back',
    'block_offensive',
    'foul_committed_advantage',
    'foul_won_advantage',
    'pass_miscommunication',
    'dribble_nutmeg',
    'pass_deflected',
    'block_deflection',
    'pass_outswinging',
    'clearance_other',
    'bad_behaviour_card_id',
    'bad_behaviour_card_name',
    'foul_committed_type_id',
    'foul_committed_type_name',
    'dribble_overrun',
    'block_save_block',
    'miscontrol_aerial_won',
    'shot_aerial_won',
    'foul_committed_offensive',
    'pass_straight',
    'ball_recovery_offensive',
    'shot_deflected',
    'shot_open_goal',
    'goalkeeper_punched_out',
    'foul_committed_penalty',
    'foul_won_penalty',
    'shot_shot_shot_assist',
    'dribble_no_touch',
    'shot_saved_to_post',
    'goalkeeper_shot_saved_to_post',
    'shot_shot_goal_assist',
    'shot_follows_dribble',
    'goalkeeper_lost_out',
    'goalkeeper_lost_in_play',
    'player_off_permanent',
    'shot_redirect',
    'goalkeeper_success_out',
    'goalkeeper_success_in_play',
    'x',
    'y',
    'end_x',
    'end_y',
    'carry_end_x',
    'carry_end_y',
    'goalkeeper_end_x',
    'goalkeeper_end_y',
    'shot_end_x',
    'shot_end_y',
    'shot_end_z',
    'prog_pass',
    'prog_carry',
    'pass_or_carry_angle',
    'phase',
    'count_hi',
    'count_hsr',
    'count_high_acceleration',
    'count_high_deceleration',
    'count_medium_acceleration',
    'count_medium_deceleration',
    'count_sprint',
    'high_intensity_hi_distance',
    'high_speed_running_hsr_distance',
    'm/min',
    'max_speed',
    'running_distance',
    'sprinting_distance',
    'total_distance']
    dfxT.drop(columns=columns_to_drop, inplace=True)
    df = df.merge(dfxT, on='index', how='left')

    def find_shot_location(row, df):
        if row['type_name'] != 'Shot':
            return pd.Series([None, None, None])
        
        shot_index = row.name  # Get the row's index
        possession_id = row['possession']
        
        # Look for previous events in the same possession
        prev_events = df.loc[:shot_index - 1]
        prev_in_possession = prev_events[prev_events['possession'] == possession_id]

        # Look for the last Ball Receipt* event
        last_receipt = prev_in_possession[prev_in_possession['type_name'] == 'Ball Receipt*']
        
        if last_receipt.empty:
            return pd.Series([None, None, None])
        
        last_receipt_row = last_receipt.iloc[-1]
        x = last_receipt_row['x']
        y = last_receipt_row['y']
        
        in_box = (
            (x is not None and y is not None) and
            (
                # Right-side penalty box
                (102 <= x <= 120 and 18 <= y <= 62) or
                # Left-side penalty box
                (0 <= x <= 18 and 18 <= y <= 62)
            )
        )
        return pd.Series([x, y, in_box])
    df[['Shoter_x', 'Shoter_y', 'in_box']] = df.apply(lambda row: find_shot_location(row, df), axis=1)
    output_folder = r"E:\Ai_com\match_enriched\new"

    print(df[df['type_name'] == 'Shot'][['player_name', 'Shoter_x', 'Shoter_y', 'in_box','shot_type_name',]].head(10))
    df.to_csv(os.path.join(output_folder, match_id), index=False)
    

    def players_passing_dfs(pname):
        # Filter for passes by the player
        dfpass = df[(df['type_name'] == 'Pass') & (df['player_name'] == pname)]
        acc_pass = dfpass[dfpass['pass_outcome_name'] == "1"]
        iac_pass = dfpass[dfpass['pass_outcome_name'] == 'Incomplete']
        
        # Calculate accurate pass percentage
        if len(dfpass) != 0:
            accurate_pass_perc = round((len(acc_pass) / len(dfpass)) * 100, 2)
        else:
            accurate_pass_perc = 0
        
        # Filter for progressive passes
        pro_pass = acc_pass[
            (acc_pass['prog_pass'] >= 9.25) & 
            (acc_pass['x'] >= 41) & 
            (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for through balls
        Thr_ball = dfpass[
            (dfpass['pass_technique_name'].str.contains('Throughball', case=False, na=False)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        Thr_ball_acc = Thr_ball[Thr_ball['pass_outcome_name'] == "1"]
        
        # Filter for long balls
        Lng_ball = dfpass[
            (dfpass['pass_pass_cluster_label'].str.contains('Long', case=False, na=False)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        Lng_ball_acc = Lng_ball[Lng_ball['pass_outcome_name'] =="1"]
        
        # Filter for crosses
        Crs_pass = dfpass[
            (dfpass['pass_cross'].astype(str).str.contains('TRUE', case=False, na=False)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        Crs_pass_acc = Crs_pass[Crs_pass['pass_outcome_name'] == "1"]
        
        # Filter for key passes
        key_pass = dfpass[dfpass['pass_shot_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
        
        # Filter for pre-assists
        df_no_carry = df[df['type_name'] != 'Carry'].reset_index(drop=True)
        pre_asst = df_no_carry[
            (df_no_carry['pass_goal_assist'].shift(-1).astype(str).str.contains('TRUE', case=False, na=False)) & 
            (df_no_carry['type_name'] == 'Pass') & 
            (df_no_carry['pass_outcome_name'] == "1") & 
            (df_no_carry['player_name'] == pname)
        ]
        
        # Filter for shot buildup
        shot_buildup = df_no_carry[
            (df_no_carry['pass_shot_assist'].shift(-1).astype(str).str.contains('TRUE', case=False, na=False)) & 
            (df_no_carry['type_name'] == 'Pass') & 
            (df_no_carry['pass_outcome_name'] == "1") & 
            (df_no_carry['player_name'] == pname)
        ]
        
        # Filter for goal assists
        g_assist = dfpass[dfpass['pass_goal_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
        
        # Filter for final third passes
        fnl_thd = acc_pass[
            (acc_pass['end_x'] >= 92.5) & 
            (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for penalty box passes
        pen_box = acc_pass[
            (acc_pass['end_x'] >= 102) & 
            (acc_pass['end_y'] >= 18.62) & 
            (acc_pass['end_y'] <= 62) & 
            (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for forward passes
        frwd_pass = dfpass[
            (dfpass['pass_or_carry_angle'] >= -85) & 
            (dfpass['pass_or_carry_angle'] <= 85) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for backward passes
        back_pass = dfpass[
            ((dfpass['pass_or_carry_angle'] >= 95) & (dfpass['pass_or_carry_angle'] <= 180)) | 
            ((dfpass['pass_or_carry_angle'] >= -180) & (dfpass['pass_or_carry_angle'] <= -95)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for sideways passes
        side_pass = dfpass[
            ((dfpass['pass_or_carry_angle'] >= 85) & (dfpass['pass_or_carry_angle'] <= 95)) | 
            ((dfpass['pass_or_carry_angle'] >= -95) & (dfpass['pass_or_carry_angle'] <= -85)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for accurate forward, backward, and sideways passes
        frwd_pass_acc = frwd_pass[frwd_pass['pass_outcome_name'] == "1"]
        back_pass_acc = back_pass[back_pass['pass_outcome_name'] == "1"]
        side_pass_acc = side_pass[side_pass['pass_outcome_name'] == "1"]
        
        # Filter for corners
        corners = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Corner', case=False, na=False)]
        corners_acc = corners[corners['pass_outcome_name'] == "1"]
        
        # Filter for free kicks
        freekik = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Goal Kick', case=False, na=False)]
        freekik_acc = freekik[freekik['pass_outcome_name'] == "1"]
        
        # Filter for throw-ins
        thins = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Throw In', case=False, na=False)]
        thins_acc = thins[thins['pass_outcome_name'] == "1"]
        
        # Filter for long balls
        lngball = dfpass[
            (dfpass['pass_pass_cluster_label'].astype(str).str.contains('Long', case=False, na=False)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        lngball_acc = lngball[lngball['pass_outcome_name'] == "1"]
        
        # Calculate forward pass accuracy
        if len(frwd_pass) != 0:
            Forward_Pass_Accuracy = round((len(frwd_pass_acc) / len(frwd_pass)) * 100, 2)
        else:
            Forward_Pass_Accuracy = 0
        
        # Calculate xT metrics
        df_xT_inc = (dfpass[dfpass['xT'] > 0])['xT'].sum().round(2)
        df_xT_dec = (dfpass[dfpass['xT'] < 0])['xT'].sum().round(2)
        total_xT = dfpass['xT'].sum().round(2)
        
        # Return the results as a dictionary
        return {
            'Name': pname,
            'Total_Passes': len(dfpass),
            'Accurate_Passes': len(acc_pass),
            'Miss_Pass': len(iac_pass),
            'Passing_Accuracy': accurate_pass_perc,
            'Progressive_Passes': len(pro_pass),
            'Chances_Created': len(key_pass),
            'Assists': len(g_assist),
            'Pre-Assists': len(pre_asst),
            'Buil-up_to_Shot': len(shot_buildup),
            'Final_Third_Passes': len(fnl_thd),
            'Passes_In_Penalty_Box': len(pen_box),
            'Through_Pass_Attempts': len(Thr_ball),
            'Accurate_Through_Passes': len(Thr_ball_acc),
            'Crosses_Attempts': len(Crs_pass),
            'Accurate_Crosses': len(Crs_pass_acc),
            'Longballs_Attempts': len(lngball),
            'Accurate_Longballs': len(lngball_acc),
            'Corners_Taken': len(corners),
            'Accurate_Corners': len(corners_acc),
            'Freekicks_Taken': len(freekik),
            'Accurate_Freekicks': len(freekik_acc),
            'ThrowIns_Taken': len(thins),
            'Accurate_ThrowIns': len(thins_acc),
            'Forward_Pass_Attempts': len(frwd_pass),
            'Accurate_Forward_Pass': len(frwd_pass_acc),
            'Side_Pass_Attempts': len(side_pass),
            'Accurate_Side_Pass': len(side_pass_acc),
            'Back_Pass_Attempts': len(back_pass),
            'Accurate_Back_Pass': len(back_pass_acc),
            'xT_Increased_From_Pass': df_xT_inc,
            'xT_Decreased_From_Pass': df_xT_dec,
            'Total_xT_From_Pass': total_xT 
        }


    # Get unique player names
    pnames = df['player_name'].unique()

    # Create a list of dictionaries to store the counts for each player
    data = []

    for pname in pnames:
        counts = players_passing_dfs(pname)
        data.append(counts)

    # Convert the list of dictionaries to a DataFrame
    passing_dfs_df = pd.DataFrame(data)

    # Sort the DataFrame by 'xT_Decreased_From_Pass' in descending order
    passing_dfs_df = passing_dfs_df.sort_values(by='xT_Decreased_From_Pass', ascending=False).reset_index(drop=True)

    # Display the DataFrame
    passing_dfs_df
    def players_passing_stats(pname):
        # Filter for passes by the player
        dfpass = df[(df['type_name'] == 'Pass') & (df['player_name'] == pname)]
        acc_pass = dfpass[dfpass['pass_outcome_name'] == "1"]
        iac_pass = dfpass[dfpass['pass_outcome_name'] == 'Incomplete']
        
        # Calculate accurate pass percentage
        if len(dfpass) != 0:
            accurate_pass_perc = round((len(acc_pass) / len(dfpass)) * 100, 2)
        else:
            accurate_pass_perc = 0
        
        # Filter for progressive passes
        pro_pass = acc_pass[
            (acc_pass['prog_pass'] >= 9.25) & 
            (acc_pass['x'] >= 41) & 
            (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for through balls
        Thr_ball = dfpass[
            (dfpass['pass_technique_name'].str.contains('Throughball', case=False, na=False)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        Thr_ball_acc = Thr_ball[Thr_ball['pass_outcome_name'] == "1"]
        
        # Filter for long balls
        Lng_ball = dfpass[
            (dfpass['pass_pass_cluster_label'].str.contains('Long', case=False, na=False)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        Lng_ball_acc = Lng_ball[Lng_ball['pass_outcome_name'] =="1"]
        
        # Filter for crosses
        Crs_pass = dfpass[
            (dfpass['pass_cross'].astype(str).str.contains('TRUE', case=False, na=False)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        Crs_pass_acc = Crs_pass[Crs_pass['pass_outcome_name'] == "1"]
        
        # Filter for key passes
        key_pass = dfpass[dfpass['pass_shot_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
        
        # Filter for pre-assists
        df_no_carry = df[df['type_name'] != 'Carry'].reset_index(drop=True)
        pre_asst = df_no_carry[
            (df_no_carry['pass_goal_assist'].shift(-1).astype(str).str.contains('TRUE', case=False, na=False)) & 
            (df_no_carry['type_name'] == 'Pass') & 
            (df_no_carry['pass_outcome_name'] == "1") & 
            (df_no_carry['player_name'] == pname)
        ]
        
        # Filter for shot buildup
        shot_buildup = df_no_carry[
            (df_no_carry['pass_shot_assist'].shift(-1).astype(str).str.contains('TRUE', case=False, na=False)) & 
            (df_no_carry['type_name'] == 'Pass') & 
            (df_no_carry['pass_outcome_name'] == "1") & 
            (df_no_carry['player_name'] == pname)
        ]
        
        # Filter for goal assists
        g_assist = dfpass[dfpass['pass_goal_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
        
        # Filter for final third passes
        fnl_thd = acc_pass[
            (acc_pass['end_x'] >= 92.5) & 
            (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for penalty box passes
        pen_box = acc_pass[
            (acc_pass['end_x'] >= 102) & 
            (acc_pass['end_y'] >= 18.62) & 
            (acc_pass['end_y'] <= 62) & 
            (~acc_pass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for forward passes
        frwd_pass = dfpass[
            (dfpass['pass_or_carry_angle'] >= -85) & 
            (dfpass['pass_or_carry_angle'] <= 85) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for backward passes
        back_pass = dfpass[
            ((dfpass['pass_or_carry_angle'] >= 95) & (dfpass['pass_or_carry_angle'] <= 180)) | 
            ((dfpass['pass_or_carry_angle'] >= -180) & (dfpass['pass_or_carry_angle'] <= -95)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for sideways passes
        side_pass = dfpass[
            ((dfpass['pass_or_carry_angle'] >= 85) & (dfpass['pass_or_carry_angle'] <= 95)) | 
            ((dfpass['pass_or_carry_angle'] >= -95) & (dfpass['pass_or_carry_angle'] <= -85)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        
        # Filter for accurate forward, backward, and sideways passes
        frwd_pass_acc = frwd_pass[frwd_pass['pass_outcome_name'] == "1"]
        back_pass_acc = back_pass[back_pass['pass_outcome_name'] == "1"]
        side_pass_acc = side_pass[side_pass['pass_outcome_name'] == "1"]
        
        # Filter for corners
        corners = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Corner', case=False, na=False)]
        corners_acc = corners[corners['pass_outcome_name'] == "1"]
        
        # Filter for free kicks
        freekik = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Goal Kick', case=False, na=False)]
        freekik_acc = freekik[freekik['pass_outcome_name'] == "1"]
        
        # Filter for throw-ins
        thins = dfpass[dfpass['play_pattern_name'].astype(str).str.contains('From Throw In', case=False, na=False)]
        thins_acc = thins[thins['pass_outcome_name'] == "1"]
        
        # Filter for long balls
        lngball = dfpass[
            (dfpass['pass_pass_cluster_label'].astype(str).str.contains('Long', case=False, na=False)) & 
            (~dfpass['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        lngball_acc = lngball[lngball['pass_outcome_name'] == "1"]
        
        # Calculate forward pass accuracy
        if len(frwd_pass) != 0:
            Forward_Pass_Accuracy = round((len(frwd_pass_acc) / len(frwd_pass)) * 100, 2)
        else:
            Forward_Pass_Accuracy = 0
        
        # Calculate xT metrics
        df_xT_inc = (dfpass[dfpass['xT'] > 0])['xT'].sum().round(2)
        df_xT_dec = (dfpass[dfpass['xT'] < 0])['xT'].sum().round(2)
        total_xT = dfpass['xT'].sum().round(2)
        
        # Return the results as a dictionary
        return {
            'Name': pname,
            'Total_Passes': len(dfpass),
            'Accurate_Passes': len(acc_pass),
            'Miss_Pass': len(iac_pass),
            'Passing_Accuracy': accurate_pass_perc,
            'Progressive_Passes': len(pro_pass),
            'Chances_Created': len(key_pass),
            'Assists': len(g_assist),
            'Pre-Assists': len(pre_asst),
            'Buil-up_to_Shot': len(shot_buildup),
            'Final_Third_Passes': len(fnl_thd),
            'Passes_In_Penalty_Box': len(pen_box),
            'Through_Pass_Attempts': len(Thr_ball),
            'Accurate_Through_Passes': len(Thr_ball_acc),
            'Crosses_Attempts': len(Crs_pass),
            'Accurate_Crosses': len(Crs_pass_acc),
            'Longballs_Attempts': len(lngball),
            'Accurate_Longballs': len(lngball_acc),
            'Corners_Taken': len(corners),
            'Accurate_Corners': len(corners_acc),
            'Freekicks_Taken': len(freekik),
            'Accurate_Freekicks': len(freekik_acc),
            'ThrowIns_Taken': len(thins),
            'Accurate_ThrowIns': len(thins_acc),
            'Forward_Pass_Attempts': len(frwd_pass),
            'Accurate_Forward_Pass': len(frwd_pass_acc),
            'Side_Pass_Attempts': len(side_pass),
            'Accurate_Side_Pass': len(side_pass_acc),
            'Back_Pass_Attempts': len(back_pass),
            'Accurate_Back_Pass': len(back_pass_acc),
            'xT_Increased_From_Pass': df_xT_inc,
            'xT_Decreased_From_Pass': df_xT_dec,
            'Total_xT_From_Pass': total_xT 
        }


    # Get unique player names
    pnames = df['player_name'].unique()

    # Create a list of dictionaries to store the counts for each player
    data = []

    for pname in pnames:
        counts = players_passing_stats(pname)
        data.append(counts)

    # Convert the list of dictionaries to a DataFrame
    passing_stats_df = pd.DataFrame(data)

    # Sort the DataFrame by 'pr_count' in descending order
    passing_stats_df = passing_stats_df.sort_values(by='xT_Decreased_From_Pass', ascending=False).reset_index(drop=True)
    passing_stats_df
    def player_carry_stats(pname):
        df_carry = df[(df['type_name']=='Carry') & (df['player_name']==pname)]
        led_shot1 = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['shot_outcome_name'].shift(-1)==('Saved'))]
        led_shot2 = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['shot_outcome_name'].shift(-1)==('Shot'))]
        led_shot = pd.concat([led_shot1, led_shot2])
        led_goal1 = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['pass_goal_assist'].shift(-1)==('TRUE'))]
        led_goal2 = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['shot_outcome_name'].shift(-1)=='Goal')]
        led_goal = pd.concat([led_goal1, led_goal2])
        pro_carry = df_carry[(df_carry['prog_carry']>=9.25) & (df_carry['x']>=42)]
        fth_carry = df_carry[(df_carry['x']<70) & (df_carry['carry_end_x']>=92.5)]
        box_entry = df_carry[(df_carry['carry_end_x']>=102) & (df_carry['carry_end_y']>=18) & (df_carry['carry_end_y']<=62) &
                    ~((df_carry['x']>=102) & (df_carry['y']>=18) & (df_carry['y']<=62))]
        disp = df[(df['type_name']=='Carry') & (df['player_name']==pname) & (df['type_name'].shift(-1)=='Dispossessed')]
        df_to = df[(df['type_name']=='Dribble') & (df['player_name']==pname)]
        t_ons = df_to[df_to['dribble_outcome_name']=='Complete']
        t_onu = df_to[df_to['dribble_outcome_name']=='Incomplete']
        df_xT_inc = df_carry[df_carry['xT']>0]
        df_xT_dec = df_carry[df_carry['xT']<0]
        total_xT = df_carry['xT'].sum().round(2)
        df_carry = df_carry.copy()
        df_carry.loc[:, 'Length'] = np.sqrt((df_carry['x'] - df_carry['carry_end_x'])**2 + (df_carry['y'] - df_carry['carry_end_y'])**2)
        median_length = round(df_carry['Length'].median(),2)
        total_length = round(df_carry['Length'].sum(),2)
        if len(df_to)!=0:
            success_rate = round((len(t_ons)/len(df_to))*100, 2)
        else:
            success_rate = 0
        
        return {
            'player_name': pname,
            'Total_Carries': len(df_carry),
            'Progressive_Carries': len(pro_carry),
            'Carries_Led_to_Shot': len(led_shot),
            'Carries_Led_to_Goal': len(led_goal),
            'Carrier_Dispossessed': len(disp),
            'Carries_Into_Final_Third': len(fth_carry),
            'Carries_Into_Penalty_Box': len(box_entry),
            'Avg._Carry_Length': median_length,
            'Total_Carry_Length': total_length,
            'xT_Increased_From_Carries': df_xT_inc['xT'].sum().round(2),
            'xT_Decreased_From_Carries': df_xT_dec['xT'].sum().round(2),
            'Total_xT_From_Carries': total_xT,
            'TakeOn_Attempts': len(df_to),
            'Successful_TakeOns': len(t_ons)
        }

    pnames = df['player_name'].unique()

    # Create a list of dictionaries to store the counts for each player
    data = []

    for pname in pnames:
        counts = player_carry_stats(pname)
        data.append(counts)

    # Convert the list of dictionaries to a DataFrame
    carrying_stats_df = pd.DataFrame(data)

    # Sort the DataFrame by 'pr_count' in descending order
    carrying_stats_df = carrying_stats_df.sort_values(by='Total_Carries', ascending=False).reset_index(drop=True)
    carrying_stats_df
    def player_shooting_stats(pname):
        # goal = df[(df['player_name']==pname) & (df['type_name']=='Goal') & (~df['type_name'].str.contains('BigChance'))]
        # miss = df[(df['player_name']==pname) & (df['type_name']=='MissedShots') & (~df['type_name'].str.contains('BigChance'))]
        # save = df[(df['player_name']==pname) & (df['type_name']=='SavedShot') & (~df['type_name'].str.contains('BigChance'))]
        # post = df[(df['player_name']==pname) & (df['type_name']=='ShotOnPost') & (~df['type_name'].str.contains('BigChance'))]
        shots_df=df

        op_sh = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_type_name'].str.contains('Open Play'))]

        goal = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Goal') & 
                        (shots_df['type_name']!='Own Goal For')]
        miss = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Off T')]
        save = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Saved')]
        blok = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Blocked')]
        post = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Post')]


        shots = df[(df['player_name']==pname) & 
            ((df['shot_outcome_name']=='Goal') | 
                (df['shot_outcome_name']=='Off T') | 
                (df['shot_outcome_name']=='Saved') | 
                (df['shot_outcome_name']=='Post') | 
                (df['shot_outcome_name']=='Blocked'))]
        shots = shots.copy()
        shots.loc[:, 'Length'] = np.sqrt((shots['Shoter_x'] - 120)**2 + (shots['Shoter_y'] - 40)**2)
        avg_dist = round(shots['Length'].mean(), 2)
        xG = shots_df[(shots_df['player_name']==pname)]['shot_statsbomb_xg'].sum().round(2)
        xGOT = shots_df[(shots_df['player_name']==pname)]['shot_gk_save_difficulty_xg'].sum().round(2)
        in_box = shots[shots['in_box'] == True]


        return {
            'player_name': pname, 
            'Total_Shots': len(shots),
            'Open_Play_Shots': len(op_sh),
            'Goals': len(goal),
            'Shot_On_Post': len(post),
            'Shot_On_Target': len(save),
            'Shot_Off_Target': len(miss),
            'Shot_Blocked': len(blok),
            'Shots_From_Inside_The_Box': len(in_box),
            'Shots_From_Outside_The_Box': len(shots) - len(in_box),
            'Avg._Shot_Distance': avg_dist,
            'xG': xG,
            'xGOT': xGOT
        }

    pnames = df['player_name'].unique()

    # Create a list of dictionaries to store the counts for each player
    data = []

    for pname in pnames:
        counts = player_shooting_stats(pname)
        data.append(counts)

    # Convert the list of dictionaries to a DataFrame
    shooting_stats_df = pd.DataFrame(data)

    # Sort the DataFrame by 'pr_count' in descending order
    shooting_stats_df = shooting_stats_df.sort_values(by='Total_Shots', ascending=False).reset_index(drop=True)
    shooting_stats_df
    def player_shooting_stats(pname):
        # goal = df[(df['player_name']==pname) & (df['type_name_name']=='Goal') & (~df['type_name_name'].str.contains('BigChance'))]
        # miss = df[(df['player_name']==pname) & (df['type_name_name']=='MissedShots') & (~df['type_name_name'].str.contains('BigChance'))]
        # save = df[(df['player_name']==pname) & (df['type_name_name']=='SavedShot') & (~df['type_name_name'].str.contains('BigChance'))]
        # post = df[(df['player_name']==pname) & (df['type_name_name']=='ShotOnPost') & (~df['type_name_name'].str.contains('BigChance'))]
        shots_df=df

        op_sh = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_type_name'].str.contains('Open Play'))]

        goal = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Goal') & 
                        (shots_df['type_name']!='Own Goal For')]
        miss = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Off T')]
        save = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Saved')]
        blok = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Blocked')]
        post = shots_df[(shots_df['player_name']==pname) & (shots_df['shot_outcome_name']=='Post')]


        shots = df[(df['player_name']==pname) & 
            ((df['shot_outcome_name']=='Goal') | 
                (df['shot_outcome_name']=='Off T') | 
                (df['shot_outcome_name']=='Saved') | 
                (df['shot_outcome_name']=='Post') | 
                (df['shot_outcome_name']=='Blocked'))]
        shots = shots.copy()
        shots.loc[:, 'Length'] = np.sqrt((shots['Shoter_x'] - 120)**2 + (shots['Shoter_y'] - 40)**2)
        avg_dist = round(shots['Length'].mean(), 2)
        xG = shots_df[(shots_df['player_name']==pname)]['shot_statsbomb_xg'].sum().round(2)
        xGOT = shots_df[(shots_df['player_name']==pname)]['shot_gk_save_difficulty_xg'].sum().round(2)
        in_box = shots[shots['in_box'] == True]


        return {
            'player_name': pname, 
            'Total_Shots': len(shots),
            'Open_Play_Shots': len(op_sh),
            'Goals': len(goal),
            'Shot_On_Post': len(post),
            'Shot_On_Target': len(save),
            'Shot_Off_Target': len(miss),
            'Shot_Blocked': len(blok),
            'Shots_From_Inside_The_Box': len(in_box),
            'Shots_From_Outside_The_Box': len(shots) - len(in_box),
            'Avg._Shot_Distance': avg_dist,
            'xG': xG,
            'xGOT': xGOT
        }

    pnames = df['player_name'].unique()

    # Create a list of dictionaries to store the counts for each player
    data = []

    for pname in pnames:
        counts = player_shooting_stats(pname)
        data.append(counts)

    # Convert the list of dictionaries to a DataFrame
    shooting_stats_df = pd.DataFrame(data)

    # Sort the DataFrame by 'pr_count' in descending order
    shooting_stats_df = shooting_stats_df.sort_values(by='Total_Shots', ascending=False).reset_index(drop=True)
    shooting_stats_df
    def player_pass_receiving_stats(pname):
        # Step 1: Get all successful passes
        successful_passes = df[(df['type_name'] == 'Pass') & (df['pass_outcome_name'] == '1')]

        # Step 2: Identify passes where the next row is received by `pname`
        pass_indexes = successful_passes.index
        next_rows = df.loc[pass_indexes + 1]
        received_passes = successful_passes[next_rows['player_name'].values == pname]
        
        # This is the reception index (row after the pass)
        reception_indexes = received_passes.index + 1
        receptions = df.loc[reception_indexes].copy()
        receptions['x'] = received_passes['end_x'].values
        receptions['y'] = received_passes['end_y'].values

        # Step 3: Track receiver's next action
        next_action_indexes = reception_indexes + 1
        next_actions = df.loc[next_action_indexes].copy()

        # Step 4: Ball retention = not losing the ball in next action
        ball_retain = next_actions[
            (next_actions['type_name'] != 'Foul Committed') &
            (next_actions['type_name'] != 'Dispossessed')
        ]

        if len(next_actions) != 0:
            ball_retention = round((len(ball_retain) / len(next_actions)) * 100, 2)
        else:
            ball_retention = 0

        # Step 5: Now apply filters using the pass that led to these receptions
        dfp = received_passes.copy()

        dfkp = dfp[dfp['pass_shot_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
        dfas = dfp[dfp['pass_goal_assist'].astype(str).str.contains('TRUE', case=False, na=False)]
        dfnt = dfp[dfp['end_x'] >= 92]
        dfpen = dfp[(dfp['end_x'] >= 102) & (dfp['end_y'] >= 18) & (dfp['end_y'] <= 62)]
        dfpro = dfp[
            (dfp['prog_pass'] >= 9.25) &
            (dfp['x'] >= 41) &
            (~dfp['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        dfcros = dfp[
            (dfp['pass_cross'].astype(str).str.contains('TRUE', case=False, na=False)) &
            (~dfp['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        dfxT = dfp[dfp['xT'] >= 0]
        dflb = dfp[
            (dfp['pass_pass_cluster_label'].astype(str).str.contains('Long', case=False, na=False)) &
            (~dfp['play_pattern_name'].astype(str).str.contains('From Corner|From Goal Kick', case=False, na=False))
        ]
        cutback = dfp[
            ((dfp['x'] >= 100.5) & (dfp['x'] <= 120) &
            (((dfp['y'] >= 46.6) & (dfp['y'] <= 64)) | ((dfp['y'] >= 16) & (dfp['y'] <= 32))) &
            (dfp['end_y'] >= 32) & (dfp['end_y'] <= 46.5) & (dfp['end_x'] >= 93.3))
        ]

        # Most passes received from the same player
        if len(dfp) > 0:
            top_sender = dfp['player_name'].value_counts().idxmax()
            top_sender_count = dfp['player_name'].value_counts().max()
            most_from = f'{top_sender}({top_sender_count})'
        else:
            most_from = 'None(0)'

        return {
            'Name': pname,
            'Total_Passes_Received': len(dfp),
            'Key_Passes_Received': len(dfkp),
            'Assists_Received': len(dfas),
            'Progressive_Passes_Received': len(dfpro),
            'Passes_Received_In_Final_Third': len(dfnt),
            'Passes_Received_In_Opponent_Penalty_Box': len(dfpen),
            'Crosses_Received': len(dfcros),
            'Longballs_Received': len(dflb),
            'Cutbacks_Received': len(cutback),
            'Ball_Retention': ball_retention,
            'xT_Received': round(dfxT['xT'].sum(), 2),
            'Avg._Distance_Of_Pass_Receiving_Form_Oppo._Goal_Line': round(120 - dfp['end_x'].median(), 2) if not dfp.empty else None,
            'Most_Passes_Received_From_(Count)': most_from
    
        }

    pnames = df['player_name'].unique()

    # Create a list of dictionaries to store the counts for each player
    data = []

    for pname in pnames:
        counts = player_pass_receiving_stats(pname)
        data.append(counts)

    # Convert the list of dictionaries to a DataFrame
    player_pass_receiving_stats_df = pd.DataFrame(data)

    # Sort the DataFrame by 'pr_count' in descending order
    player_pass_receiving_stats_df = player_pass_receiving_stats_df.sort_values(by='Total_Passes_Received', ascending=False).reset_index(drop=True)
    player_pass_receiving_stats_df
    def player_defensive_stats(pname):

        playerdf = df[df["player_name"] == pname]

        ball_wins = playerdf[
            (playerdf["type_name"] == 'Interception') | (playerdf["type_name"] == 'Ball Recovery')
        ]

        f_third = ball_wins[ball_wins['x'] >= 80]
        m_third = ball_wins[(ball_wins['x'] > 40) & (ball_wins['x'] < 80)]
        d_third = ball_wins[ball_wins['x'] <= 40]

        tk = playerdf[
            (playerdf["type_name"] == 'Duel') &
            (playerdf['duel_type_name'].str.contains('Tackle', case=False, na=False))
        ]

        tk_u = playerdf[
            (playerdf["type_name"] == 'Duel') &
            (playerdf['duel_type_name'].str.contains('Tackle', case=False, na=False)) &
            (playerdf['duel_outcome_name'].isin(['Lost In Play', 'Lost']))
        ]

        intc = df[
        (df['player_name'] == pname) &
        (df['type_name'] == 'Interception') &
        (df['interception_outcome_name'].isin(['Success In Play', 'Won']))
    ]


        br = df[
        (df['player_name'] == pname) &
        (df['type_name'] == 'Ball Recovery') &
        (df['ball_recovery_recovery_failure'].astype(str) != 'True')
    ]


        cl = playerdf[playerdf["type_name"] == 'Clearance']
        fl = playerdf[playerdf["type_name"] == 'Foul Committed']

        ar = playerdf[
            (playerdf["type_name"] == 'Duel') &
            (playerdf['duel_type_name'].str.contains('Aerial', case=False, na=False))
        ]

        ar_u = playerdf[
            (playerdf["type_name"] == 'Duel') &
            (playerdf['duel_type_name'].str.contains('Aerial', case=False, na=False)) &
            (playerdf['duel_outcome_name'] == 'Aerial Lost')
        ]

        pass_bl = playerdf[playerdf["type_name"] == 'Block']

        shot_bl = playerdf[
            (playerdf["type_name"] == 'Block') &
            (playerdf['block_save_block'].astype(str).str.contains('TRUE', case=False, na=False))
        ]

        drb_pst = playerdf[playerdf["type_name"] == 'Dribbled Past']

        drb_tkl = df[
            (df["player_name"] == pname) &
            (df["type_name"] == 'Duel') &
            (df['duel_type_name'].str.contains('Tackle', case=False, na=False)) &
            (df["duel_outcome_name"].isin(['Lost In Play', 'Lost'])) &
            (df["type_name"].shift(1) == 'Dribble')
        ]
        # 1. Identify all defensive failures by the player
        errors = df[
            (df['player_name'] == pname) & (
                # Unsuccessful tackle or duel
                ((df['type_name'] == 'Duel') &
                (df['duel_type_name'].str.contains('Tackle|Aerial', case=False, na=False)) &
                (df['duel_outcome_name'].isin(['Lost In Play', 'Lost', 'Aerial Lost']))) |

                # Interception attempt that doesn't stop the ball (can add more logic if needed)

                # Fouls committed
                (df['type_name'] == 'Foul Committed')
            )
        ]

        # 2. Initialize counters
        err_lat = 0  # Error leading to shot
        err_lgl = 0  # Error leading to goal

        # 3. Loop through those events and check the next 1-2 events
        for idx in errors.index:
            next_events = df.loc[idx + 1: idx + 2]
            if not next_events.empty and 'type_name' in next_events:
                shot_events = next_events[next_events['type_name'] == 'Shot']
                if not shot_events.empty:
                    err_lat += 1
                    if 'shot_outcome_name' in shot_events and any(shot_events['shot_outcome_name'] == 'Goal'):
                        err_lgl += 1
        return {
            'Player': pname,
            'Interceptions': len(intc),
            'Ball Recoveries': len(br),
            'Tackles': len(tk),
            'Unsuccessful Tackles': len(tk_u),
            'Clearances': len(cl),
            'Fouls Committed': len(fl),
            'Aerial Duels': len(ar),
            'Aerial Duels Lost': len(ar_u),
            'Passes Blocked': len(pass_bl),
            'Shots Blocked': len(shot_bl),
            'Dribbled Past': len(drb_pst),
            'Dribble Tackles': len(drb_tkl),
            'Ball Wins in Final Third': len(f_third),
            'Ball Wins in Middle Third': len(m_third),
            'Ball Wins in Defensive Third': len(d_third),
            'Errors Leading to Shot': err_lat,
            'Errors Leading to Goal': err_lgl
        }
    names = df['player_name'].unique()

    # Create a list of dictionaries to store the counts for each player
    data = []

    for pname in pnames:
        counts = player_defensive_stats(pname)
        data.append(counts)

    # Convert the list of dictionaries to a DataFrame
    player_defensive_stats = pd.DataFrame(data)
    player_defensive_stats
    def touches_stats(pname):
        touch_types = [
            'Ball Receipt*',"Pass"
        ]
        df_player = df[df['player_name'] == pname]
        touches = df_player[df_player['type_name'].isin(touch_types)]
        final_third = touches[touches['x'] >= 80]
        pen_box = touches[(touches['x'] >= 102) & (touches['y'] >= 18) & (touches['y'] <= 62)]


        return {
            'Name': pname,
            'Total_Touches': len(touches),
            'Touches_In_Final_Third': len(final_third),
            'Touches_In_Opponent_Penalty_Box': len(pen_box),


        }
        

    # Create a list of dictionaries to store the counts for each player
    data = []

    for pname in pnames:
        counts = touches_stats(pname)
        data.append(counts)

    # Convert the list of dictionaries to a DataFrame
    touches_stats_df = pd.DataFrame(data)

    # Sort the DataFrame by 'pr_count' in descending order

    touches_stats_df = touches_stats_df.sort_values(by='Total_Touches', ascending=False).reset_index(drop=True)
    touches_stats_df
    # Rename the column in shooting_stats_df to match passing_stats_df
    # Rename columns in all DataFrames to ensure consistency
    # Rename columns in all DataFrames to ensure consistency
    shooting_stats_df = shooting_stats_df.rename(columns={'player_name': 'Name'})
    passing_stats_df = passing_stats_df.rename(columns={'Name': 'Name'})  # Ensure 'Name' is consistent
    carrying_stats_df = carrying_stats_df.rename(columns={'player_name': 'Name'})
    player_pass_receiving_stats_df = player_pass_receiving_stats_df.rename(columns={'player_name': 'Name'})
    player_defensive_stats_df = player_defensive_stats_df.rename(columns={'player_name': 'Name'})
    touches_stats_df = touches_stats_df.rename(columns={'player_name': 'Name'})  # Rename if necessary

    # Sort touches_stats_df by 'Total_Touches' (if not already sorted)
    touches_stats_df = touches_stats_df.sort_values(by='Total_Touches', ascending=False).reset_index(drop=True)

    # Merge all DataFrames
    player_stats_df = shooting_stats_df.merge(passing_stats_df, on='Name', how='left')
    player_stats_df = player_stats_df.merge(carrying_stats_df, on='Name', how='left')
    player_stats_df = player_stats_df.merge(player_pass_receiving_stats_df, on='Name', how='left')
    player_stats_df = player_stats_df.merge(player_defensive_stats_df, on='Name', how='left')
    player_stats_df = player_stats_df.merge(touches_stats_df, on='Name', how='left')

    # Display the final DataFrame
    # 
    player_stats_df
    # Calculate total OBV contributions for each player
    player_obv_df = (
        df.groupby('player_name')[['obv_for_net', 'obv_against_net']]
        .sum()
        .reset_index()
        .rename(columns={'player_name': 'Name'})
    )

    # Optional: Add a combined OBV net value (offensive - defensive)
    player_obv_df['obv_total_net'] = player_obv_df['obv_for_net'] - player_obv_df['obv_against_net']

    player_stats_df = player_stats_df.merge(player_obv_df, on='Name', how='left')
    player_stats_df
    out_file = os.path.join(folder_path, f"match_{match_id}_player_stats.csv")
    player_stats_df.to_csv(out_file, index=False)
    print(f"✅ Processed match {match_id}")


(1518, 184)


C:\Users\lahd2\AppData\Local\Temp\ipykernel_39200\946177516.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfxT['start_zone_value_xT'] = dfxT[['x1_bin_xT', 'y1_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\lahd2\AppData\Local\Temp\ipykernel_39200\946177516.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfxT['end_zone_value_xT'] = dfxT[['x2_bin_xT', 'y2_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)


                player_name  Shoter_x  Shoter_y in_box shot_type_name
66             Mutsuki Kato      91.9      32.2  False      Open Play
276          Yoshio Koizumi     105.2      29.6   True      Open Play
280   Thiago Santos Santana     107.8      37.9   True      Open Play
316           Yusuke Matsuo      96.9      43.3  False      Open Play
471          Makoto Mitsuta      92.9      67.8  False      Open Play
517             Yuki Ohashi      68.5      45.3  False      Open Play
812         Pieros Soteriou      23.7       4.0  False      Open Play
986            Mutsuki Kato      88.6      20.8  False      Open Play
1012        Takahiro Sekine     114.4      35.4   True      Open Play
1036       Alexander Scholz     113.6      49.2   True      Open Play
✅ Processed match 3925226


KeyError: 'type_name'